In [1]:
# import libraries
import re
import pdfplumber
import json

In [2]:
# define PDF file path
file_name = 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education (2018)'
input_file = file_name + '.pdf'
output_file = file_name + '_c1_statements.jsonl'

In [3]:
pages_to_extract = list(range(87, 96))

    
all_text = ""

with pdfplumber.open(input_file) as pdf:
    for page_num, page in enumerate(pdf.pages):
        if page_num + 1 in pages_to_extract:
            page_text = page.extract_text()
            if page_text:
                all_text += '\n' + page_text
pattern1 = r"CertPrs_2015/PMP® Project Management Professional Study Guide/Joseph Phillips/86198-8/Chapter 1"
all_text=re.sub(pattern1, '', all_text)
pattern2 = r"Self Test Answers \d+"
all_text=re.sub(pattern2, '', all_text)
pattern3 = r"SELF TEST ANSWERS"
all_text=re.sub(pattern3, '', all_text)
pattern4 = r"\d+ Chapter 1 Introducing Project Management"
all_text=re.sub(pattern4, '', all_text)
pattern5 = r"01-ch01.indd \d+ 28/11/17 4:37 PM"
all_text=re.sub(pattern5, '', all_text)    
#print(all_text)

In [4]:
section = r"^\d+\.\s+([\S\s]*?)(?=\Z|^\d+\.\s+)"
# capture sections from question to question
sections = re.finditer(section, all_text, re.MULTILINE)

qnumber = r"(^\d+)\.\s+[\S\s]"
qnumbers = re.finditer(qnumber, all_text, re.MULTILINE)

question = r"\d+\.\s+([\S\s]*?)\nA\."
questions = re.finditer(question, all_text, re.MULTILINE)

choiceA = r"\nA\.([\S\s]*?)\nB\."
choiceAs = re.finditer(choiceA, all_text, re.MULTILINE)

choiceB = r"\nB\.([\S\s]*?)\nC\."
choiceBs = re.finditer(choiceB, all_text, re.MULTILINE)

choiceC = r"\nC\.([\S\s]*?)\nD\."
choiceCs = re.finditer(choiceC, all_text, re.MULTILINE)

choiceD = r"\nD\.([\S\s]*?)\n"
choiceDs = re.finditer(choiceD, all_text, re.MULTILINE)

answer_dict={}
    
answer_key_regex = r"☑\s*([ABCD])\.\s*(.*?)\n"

matches = re.findall(answer_key_regex, all_text, re.DOTALL)

i=1
for match in matches:
    answer = match[0]
    explanation = match[1].strip()
    answer_dict[str(i)]=answer
    i+=1
answer_dict

{'1': 'B',
 '2': 'A',
 '3': 'D',
 '4': 'C',
 '5': 'A',
 '6': 'B',
 '7': 'C',
 '8': 'B',
 '9': 'C',
 '10': 'B',
 '11': 'A',
 '12': 'C',
 '13': 'B',
 '14': 'D',
 '15': 'C',
 '16': 'A',
 '17': 'A',
 '18': 'B',
 '19': 'A',
 '20': 'C',
 '21': 'C',
 '22': 'A',
 '23': 'C',
 '24': 'B',
 '25': 'D'}

In [5]:
explanation_dict={}
explanation_regex = r"☑\s*[ABCD]\.\s*(.*?)\n(?:☐✗\n(.*?\n)|$)"

matches = re.findall(explanation_regex, all_text, re.DOTALL)
i=1
for match in matches:
    explanation_correct = match[0].strip()
    explanation_incorrect = match[1].strip() if match[1] else ""
    explanation_dict[str(i)]=f"{explanation_correct} {explanation_incorrect}"
    i+=1
explanation_dict

{'1': 'A project does have a definite end date; operations do not. A, C, and D are incorrect. Projects do have a definite starting date; they do create a unique',
 '2': 'Not all information in the PMBOK Guide should be applied uniformly to all projects. It is\nthe responsibility of the project management team to determine what practices are appropriate\nfor each project. B, C, and D are incorrect. They are all false statements regarding the implementation of the',
 '3': 'A risk assessment within the project is not a business value, but a project management\nactivity. Risks can be positive or negative, but the assessment of a risk is not a business value element.\nBusiness value means that the project is contributing something positive for the organization. A, B, and C are incorrect. A is incorrect because return on investment is an example of',
 '4': 'Operations are ongoing and can last for as long as the organization is in business. Projects\nare temporary; they do not go on forever. 

In [6]:
import re
import unicodedata  # Module Unicode

matchesS = re.finditer(section, all_text, re.MULTILINE)
pmpqa = []

for sectionNum, sectionMatch in enumerate(matchesS):
    matchesQN = re.findall(qnumber, sectionMatch.group(0), re.MULTILINE)
    matchesQ = re.findall(question, sectionMatch.group(0), re.MULTILINE)
    matchesA = re.findall(choiceA, sectionMatch.group(0), re.MULTILINE)
    matchesB = re.findall(choiceB, sectionMatch.group(0), re.MULTILINE)
    matchesC = re.findall(choiceC, sectionMatch.group(0), re.MULTILINE)
    matchesD = re.findall(choiceD, sectionMatch.group(0), re.MULTILINE)
    
    for matchQN, matchQ, matchA, matchB, matchC, matchD in zip(matchesQN, matchesQ, matchesA, matchesB, matchesC, matchesD):
        # Normalize Unicode characters to their closest ASCII representation
        # This removes accents and converts special characters like '\u2018' and '\u2019' to standard ASCII equivalents.
        matchQ = unicodedata.normalize('NFKD', matchQ).encode('ascii', 'ignore').decode('utf-8')
        matchA = unicodedata.normalize('NFKD', matchA).encode('ascii', 'ignore').decode('utf-8')
        matchB = unicodedata.normalize('NFKD', matchB).encode('ascii', 'ignore').decode('utf-8')
        matchC = unicodedata.normalize('NFKD', matchC).encode('ascii', 'ignore').decode('utf-8')
        matchD = unicodedata.normalize('NFKD', matchD).encode('ascii', 'ignore').decode('utf-8')
        
        qadata = {}
        qadata['source'] = 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education'
        qadata['year'] = 2018
        qadata['type'] = 'study guide'
        qadata['qid'] = matchQN
        qadata['question'] = {"stem": matchQ.replace('\n',' ').strip(),
                              "choices": [
                                  {"label": "A", "text": matchA.replace('\n',' ').strip()},
                                  {"label": "B", "text": matchB.replace('\n',' ').strip()},
                                  {"label": "C", "text": matchC.replace('\n',' ').strip()},
                                  {"label": "D", "text": matchD.replace('\n',' ').strip()}]}
    
        
        #if matchQN in answer_dict.keys():
        qadata['answerKey'] = answer_dict.get(matchQN)
        
        statements = []
        for choice, text in zip(['A', 'B', 'C', 'D'], [matchA, matchB, matchC, matchD]):
            is_correct = (answer_dict.get(matchQN) == choice)
            statements.append({
                "label": is_correct,
                "statement": matchQ.replace('\n',' ').strip() + ' ' + text.replace('\n', ' ').strip()
            })
        exp = unicodedata.normalize('NFKD', explanation_dict.get(matchQN)).encode('ascii', 'ignore').decode('utf-8')
        qadata['explanation'] = exp
        qadata['statements'] = statements
        pmpqa.append(qadata)

pmpqa


[{'source': 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education',
  'year': 2018,
  'type': 'study guide',
  'qid': '1',
  'question': {'stem': 'As a PMP candidate, you must have a firm grasp on what constitutes a project. Which one of the following is not an attribute of a project?',
   'choices': [{'label': 'A', 'text': 'Has a definite starting date'},
    {'label': 'B', 'text': 'Has no definite end date'},
    {'label': 'C', 'text': 'Creates a product, service, or result'},
    {'label': 'D', 'text': 'Requires resources'}]},
  'answerKey': 'B',
  'explanation': 'A project does have a definite end date; operations do not. A, C, and D are incorrect. Projects do have a definite starting date; they do create a unique',
  'statements': [{'label': False,
    'statement': 'As a PMP candidate, you must have a firm grasp on what constitutes a project. Which one of the following is not an attribute of a project? Has a definite starting date'},
   {'label':

In [7]:
# save the PMP-QA dataset as jsonl
with open(output_file, 'w') as fout:
    for dic in pmpqa:
        fout.write(json.dumps(dic) + '\n')

In [1]:
# import libraries
import re
import pdfplumber
import json

In [2]:
# define PDF file path
file_name = 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education (2018)'
input_file = file_name + '.pdf'
output_file = file_name + '_chapter2_statements.jsonl'

In [3]:
pages_to_extract = list(range(127, 137))

    
all_text = ""

with pdfplumber.open(input_file) as pdf:
    for page_num, page in enumerate(pdf.pages):
        if page_num + 1 in pages_to_extract:
            page_text = page.extract_text()
            if page_text:
                all_text += '\n' + page_text
pattern1 = r"CertPrs_2015/PMP® Project Management Professional Study Guide/Joseph Phillips/86198-8/Chapter \d+"
all_text=re.sub(pattern1, '', all_text)
pattern2 = r"Self Test Answers \d+"
all_text=re.sub(pattern2, '', all_text)
pattern3 = r"SELF TEST ANSWERS"
all_text=re.sub(pattern3, '', all_text)
pattern4 = r"\d+ Chapter 2 Managing Projects in Different Environments"
all_text=re.sub(pattern4, '', all_text)
pattern5 = r"02-ch02.indd \d+ 28/11/17 4:38 PM"
all_text=re.sub(pattern5, '', all_text)   
all_text='\n1. You are the project manager for the application deployment project. This project will deploy\nsoftware to 450 sites throughout North America and in Europe. Because this project will span\nmultiple countries, you must consider the limitations imposed by communications, accessing\nresources, and other concerns. These international concerns that limit your choices in the project\nare known as which of the following?\nA. Internal enterprise environmental factors\nB. External enterprise environmental factors\nC. Cultural norms\nD. Organizational process assets\n☑ A. These are internal enterprise environmental factors, because the factors are internal to\nthe organization, but they do restrict options for the project.\n☐✗\nB, C, and D are incorrect. B is incorrect because external enterprise environmental factors\nare factors outside of the organization’s control. C is incorrect because cultural norms are factors\nof the organization culture, attitude toward work, and accepted practices within the organization.\nD, organizational process assets, are things that have been created prior to the current project\nthat the project manager can leverage to manage the current project.\n\n\n\n2. You are the project manager for your organization and you’re planning a project in consideration\nof some regulations in your industry. These regulations are also known as which of the following?\nA. Constraints\nB. External enterprise environmental factors\nC. Internal enterprise environmental factors\nD. Industry organizational process assets\n☑ B. Laws and regulations are external enterprise environmental factors.\n☐✗\nA, C, and D are incorrect. A, constraints, are anything that limit your options, such as\npredefined budget or deadline. Although regulations could be considered a constraint, this isn’t the\nbest answer for the question. C is incorrect because internal enterprise environmental factors are\nfactors that affect the project decisions within the organization’s control. D, industry organizational\nprocess assets, is not a valid project management term, so this choice is incorrect.\n3. You are the project manager of the NHQ Project. Your project team consists of experts from\ndifferent departments within your organization. What type of organizational structure is this?\nA. Virtual\nB. Weak matrix\nC. Noncollocated\nD. Dedicated\n☑ B. This is an example of a matrix structure. Although the question doesn’t define whether\nthe matrix is truly strong, weak, or balanced, weak matrix is the best answer presented.\n☐✗\nA, C, and D are incorrect. A is incorrect because a virtual team describes an organizational\nnetwork approach, with a point of contact within each group represented in the project. C is\nincorrect because this isn’t a valid organizational structure type. “Noncollocated” can be used to\ndescribe a virtual project team, but this isn’t the best answer. D is also incorrect, because there’s\nno evidence that the people on this project team will be working only on this project; neither is it\na valid organizational structure type.\n4. Beth is a new project manager for her company, and she’s working with her project team to develop\nthe project management plan. Beth knows that she must rely on several different skills to make\nher first project successful. Of the following management skills, which will she use most?\nA. Leading\nB. Communicating\nC. Influencing the organization\nD. Negotiating\n\n\n\n☑ B. Communication is the key general management skill a project manager will use the most.\n☐✗\nA, C, and D are incorrect. They are necessary, but communication accounts for the majority\nof a project manager’s time.\n5. Harold is the manager of the manufacturing unit for your company, and he’ll be a key stakeholder\non your upcoming project. To get to know Harold better, you’re having lunch with him and\ndiscussing some of the goals of the upcoming project. Harold doesn’t quite understand what you\ndo as a project manager, so you explain the concept to him. Managing a project is best described\nas which one of the following?\nA. Establishing direction\nB. Functionally controlling the project team and stakeholders\nC. Consistently producing key results expected by stakeholders\nD. Motivating and inspiring the project team to produce results that are expected by project\nstakeholders\n☑ C. Managing has to do with consistently producing key results that are expected by stakeholders.\n☐✗\nA, B, and D are incorrect. A and D describe the leadership qualities a project manager must\npossess, but they are do not best describe the concept. B is incorrect because it describes the\nfunctional management position over project team members.\n6. You are the project manager of the Server Deployment Project for your company. You’re meeting\nwith the key stakeholders to gather the requirements and review the intent of the project. You know\nthat you’ll have to meet with the stakeholders throughout the project and their influence can help\nor hinder the project’s success. You’d like to create a solid communications management plan for the\nproject. Where you can access another communications management plan to use as a template?\nA. Internal enterprise environmental factors\nB. External enterprise environmental factors\nC. Organizational process assets\nD. Other current projects in the organization\n☑ C. Of all the choices presented, this answer is best. Organizational process assets include\npast project records. You can adapt a previous project’s communications management plan to fit\nthe current project.\n☐✗\nA, B, and D are incorrect. Internal and external enterprise environmental factors and other\ncurrent projects are not the best locations for communications management plans.\n\n\n\n7. Which one of the following is an example of a physical enterprise environmental factor that could\naffect a project?\nA. Weather\nB. Regulations\nC. PMO\nD. Past project files\n☑ A. Weather is an example of a physical enterprise environmental factor. The environment in\nwhich the project is taking place is a physical enterprise environmental factor.\n☐✗\nB, C, and D are incorrect. B and C are incorrect because regulations and PMOs are not physical\nenterprise environmental factors. D is incorrect because past project files are an example of an\norganizational process asset.\n8. You are the project manager for your organization. Influencing your organization requires which\nof the following?\nA. An understanding of the organizational budget\nB. Research and documentation of proven business cases\nC. An understanding of formal and informal organizational systems\nD. Positional power\n☑ C. To influence an organization (to get things done), a project manager must understand the\nexplicit and implied organizational system.\n☐✗\nA, B, and D are incorrect. A is incorrect because the project manager may not even have\naccess to an organizational budget. B is incorrect because a proven business case may not map\nto every scenario when influencing an organization. D is incorrect because positional power may\nrelate only to a small portion of an organization, not to multiple facets of influence.\n9. Management has asked you, the project manager, to create a method to store your project\ninformation electronically. Your project records will help future project teams to make better\ndecisions and can also be used to support the product that your project is creating. What does\nmanagement want to create for your project?\nA. Organizational process assets\nB. Organizational knowledge repository\nC. Project database\nD. Project management plan\n☑ B. Management wants you to create an organizational knowledge repository. Ideally, this\nwould already exist in an organization, but the organization must start somewhere.\n☐✗\nA, C, and D are incorrect. A is incorrect because organizational process assets are things that\nhave already been created, such as forms and templates. C is incorrect because a project database\nis only one form of knowledge repository. D may be a tempting answer, but project management\nplan isn’t the best choice because this is needed for every project and won’t necessarily be an\nelectronic database of information that can be accessed by others projects.\n\n\n\n10. What does an organizational system accomplish for projects within any organizational structure?\nA. Defines the roles and responsibilities of each project team member.\nB. Provides structure and governance for the project manager.\nC. Identifies regulations and standards that the project manager must adhere to.\nD. Provides accountability for the project manager, the project team, and project stakeholders.\n☑ B. Of all the choices presented, B is the best answer because organizational systems do\nprovide structure and governance for the project manager and how the project manager leads\nand manages the project team and project work.\n☐✗\nA, C, and D are incorrect. A is incorrect because the organizational system doesn’t define the\nroles and responsibilities of the project team members. C is incorrect—standards and regulations\nare not defined by the organizational system. D is incorrect because the organizational structure\ndoesn’t provide accountability to the project manager, project team, or stakeholders.\n11. You are the project manager of the JHK Project and working with the IT department and the\naccounting department on some project requirements. There is a relationship and dynamic between\nIT and accounting that is likely to affect the project requirements. What is this relationship between\nthese two components called?\nA. Business links\nB. Matrix\nC. System dynamics\nD. Functional structuring\n☑ C. System dynamics is a way of describing the relationship between the components, such as\nthe relationship between the IT and accounting departments.\n☐✗\nA, B, and D are incorrect. These are not valid descriptions of the relationship between the\ncomponents of the system.\n12. An organization has created a program for a new endeavor. The program manager is establishing\nall of the rules, policies, and approved approaches for project management within the program.\nWhat term best describes the rules, policies, and approved project management approaches that\nthe program manager is creating?\nA. Framework\nB. Program governance\nC. Constraints\nD. Program threshold management\n☑ B. The program manager is creating the program governance for all projects that operate\nwithin the program.\n☐✗\nA, C, and D are incorrect. A is incorrect because, although frameworks can establish the\nboundaries of the projects, the better choice is program governance. C is incorrect because constraints\ndescribe things that limit the project manager’s options, such as a deadline or predetermined budget.\nD is incorrect because program threshold management isn’t a valid project management term.\n\n\n\n13. You are the project manager for the ERP Project. Your organization uses a PMO. The primary\npurpose of a project office is to do which of the following?\nA. Support the project manager\nB. Support the project sponsor\nC. Support the project team\nD. Identify the stakeholder objectives\n☑ A. The PMO supports the project manager.\n☐✗\nB, C, and D are incorrect. B and C are incorrect because the project office does not support\nthe project sponsor and project team. D is incorrect because stakeholder objectives can vary from\nstakeholder to stakeholder on a project.\n14. An organization has multiple departments, and within each department is an IT structure that\nsupports only that department. IT projects are launched within one department and not all\ndepartments in the organization as the whole. What type of organizational structure is this?\nA. Multidivisional structure\nB. Functional structure\nC. Hybrid structure\nD. Strong matrix structure\n☑ A. In this multidivisional organization, there will likely be replication of functions for each\ndivision.\n☐✗\nB, C, and D are incorrect. B is incorrect because functional structures are project-centric\nto each function, but there likely is not a replication of functions across the organization. In a\ntrue functional structure, there is not a replication of function, such as an IT group within each\ndepartment, but rather just an IT department within the organization. C is incorrect because a\nhybrid structure is blend of organizational structure types. D is incorrect because strong matrix\nstructures utilize resources from across the organization, rather than isolate them within each\ndepartment.\n15. You are the project manager of a project. The project is nearing its completion and the project\nteam is experiencing some anxiety as to what their next project will be. What type of organizational\nstructure are you operating in?\nA. Matrix\nB. Project-oriented\nC. Functional\nD. Multidivisional\n\n\n\n☑ B. In a project-oriented structure, the project team works on one project full time. Near the end\nof the project the team members may experience some anxiety as to what their next project will be.\n☐✗\nA, C, and D are incorrect. These organizational structures allow the project team members to\nreturn to their day-to-day operations after the project is completed.\n16. What type of project management office manages and controls all projects within the organization?\nA. Controlling project management office\nB. Supportive project management office\nC. Directive project management office\nD. Functional project management office\n☑ C. A directive project management office manages and controls all projects within the\norganization. The PMO control is considered high.\n☐✗\nA, B, and D are incorrect. Controlling project management offices define project governance\nthrough project management frameworks, templates, forms, project management activities, and\ncommunications. Supportive project management offices act as a consultative role by offering\nadvice, best practices, lessons learned, forms and software, and project information from similar\nprojects. D is incorrect because there is no such thing as a functional project management\noffice type.\n17. Beth is a project manager in small startup company. The project team is loosely organized and\npeople chip in to help however they can, based on their skills and the work that needs to be completed.\nThe organization owner, Sarah, has control over the project resources. What structure is Beth likely\noperating in?\nA. Organic\nB. Functional\nC. Weak matrix\nD. Strong matrix\n☑ A. An organic or simple organizational structure means that the work groups within\nthe organization are flexible. People work alongside one another regardless of their roles in the\norganization, and the project manager may have little to no authority over the project resources.\n☐✗\nB, C, and D are incorrect. B is incorrect because functional structures are based on functions\nand don’t use resources from across the organization. C and D are incorrect because matrix\nstructures do use resources from across the organization, but the structure is more clearly\ndefined than in an organic structure. Also, in a strong matrix, the project manager, rather than\nthe functional manager, will have authority over the project resources.\n\n\n\n18. An organization that typically operates as a weak matrix has decided that for a high-priority project,\nthe team will work on the project full time. In addition, the project manager will manage just this\none project full time. What type of organization structure is this?\nA. Simple\nB. Project-oriented\nC. Strong matrix\nD. Hybrid\n☑ D. This organization is a hybrid because it’s shifting from its usual weak matrix structure to\na project-oriented structure for just this one project.\n☐✗\nA, B, and C are incorrect. A is incorrect because organic or simple organizational structures\nallow individuals within the organization to be flexible and work alongside one another regardless\nof their roles in the organization. B is a tempting choice, but the organization isn’t project-oriented\nall the time, just for this one project. C, strong matrix, isn’t the best answer because this structure\nutilizes resources from all over the organization and people don’t always work on just one project.\n19. Henry, the project manager of the MHB Project, is operating within a multidivisional organizational\nstructure. Who will make decisions about the project budget?\nA. Henry\nB. Functional manager\nC. Program manager\nD. Portfolio manager\n☑ B. In a multidivisional structure, the functional manager makes decisions about the project\nbudget.\n☐✗\nA, C, and D are incorrect. The project manager, program manager, and portfolio manager\naren’t making decisions about the budget.\n20. A company has hired you as a project manager to lead a new software development project. You\nhave an assigned budget and several milestones in the project. The project sponsor has asked\nyou to wait on launching the project execution a month or two because of some unsettling news\nin the marketplace. The marketplace conditions are best described as which one of the following?\nA. High costs and high demand for resources\nB. External organizational process assets\nC. Internal enterprise environmental factors\nD. External enterprise environmental factors\n☑ D. Marketplace conditions are external enterprise environmental factors that can affect\nproject decisions in the organization.\n☐✗\nA, B, and C are incorrect. High costs and high demand for resources isn’t a good answer\nfor this question. There is no category of external organizational process assets. Marketplace\nconditions are outside of the organization, so they are considered external enterprise environmental\nfactors, rather than internal enterprise environmental factors.\n\n\n\n21. All the following are organizational process assets that can be used within a project except for\nwhich one?\nA. Templates for project documents\nB. Historical information\nC. Regulations\nD. Performance measurements\n☑ C. Regulations are an example of external enterprise environmental factors, not organizational\nprocess assets.\n☐✗\nA, B, and D are incorrect. Templates, historical information, and performance measurements\nare all examples of organizational process assets.\n22. Tracey is the project manager of the KHG Project. Her organization is a classic functional\nenvironment. Her level of authority as a project manager can be best described as which of the\nfollowing?\nA. Low\nB. Moderate\nC. Balanced\nD. High\n☑ A. Tracey will most likely have a low amount of authority in a functional organizational\nstructure.\n☐✗\nB, C, and D are incorrect. B and C are incorrect because they describe matrix structures.\nD is incorrect because it is relevant to a project-oriented structure.\n23. Project team members are most likely to work full time on a project in which of the following\norganizational structures?\nA. Functional\nB. Weak matrix\nC. Organizational\nD. Project-oriented\n☑ D. Project-oriented structures often have project team members assigned to the project on\na full-time basis.\n☐✗\nA, B, and C are incorrect. A and B are incorrect because these structures are part-time\nproject teams. C is not a valid organization structure.\n\n\n\n24. A project manager is operating in a weak matrix. She has asked management for information\nfrom a past project that is similar to her project. Where is the best place for the project manager\nto find this information?\nA. Organizational knowledge repository\nB. Organizational process assets\nC. Internal enterprise environmental factors\nD. From the project manager of the past project\n☑ A. A knowledge repository will have the information needed from the past project and is the\nbest place to locate the information.\n☐✗\nB, C, and D are incorrect. These choices don’t reflect the best places to find the knowledge.\nThough the knowledge repository could be an example of an organizational process asset,\norganizational process assets isn’t the best choice for this question. Historical information isn’t\npart of enterprise environmental factors. And although it’s logical to consult with the previous\nproject manager, this isn’t the best place to find the information needed.\n25. Stacey is the project manager of the GBN Project for her company. She’ll be using several templates\nfor a project, but she’s not certain where these templates should originate. Where can a project\nmanager usually expect to receive templates?\nA. Commercial databases\nB. The project management office\nC. The project sponsor\nD. PMIS\n☑ B. The project management office is the best choice, since its role is to support the project\nmanager.\n☐✗\nA, C, and D are incorrect. A, commercial databases, may be feasible, but it is not the best\nchoice presented. Project sponsors, C, are not typically going to provide the project manager with\ntemplates. D, project management information systems, may have project templates available, but\nthe project management office is the best choice presented.\n'
#print(all_text)

In [4]:
section = r"^\d+\.\s+([\S\s]*?)(?=\Z|^\d+\.\s+)"
# capture sections from question to question
sections = re.finditer(section, all_text, re.MULTILINE)

qnumber = r"(^\d+)\.\s+[\S\s]"
qnumbers = re.finditer(qnumber, all_text, re.MULTILINE)

question = r"\d+\.\s+([\S\s]*?)\nA\."
questions = re.finditer(question, all_text, re.MULTILINE)

choiceA = r"\nA\.([\S\s]*?)\nB\."
choiceAs = re.finditer(choiceA, all_text, re.MULTILINE)

choiceB = r"\nB\.([\S\s]*?)\nC\."
choiceBs = re.finditer(choiceB, all_text, re.MULTILINE)

choiceC = r"\nC\.([\S\s]*?)\nD\."
choiceCs = re.finditer(choiceC, all_text, re.MULTILINE)

choiceD = r"\nD\.([\S\s]*?)\n"
choiceDs = re.finditer(choiceD, all_text, re.MULTILINE)

answer_dict={}
    
answer_key_regex = r"☑\s*([ABCD])\.\s*(.*?)\n"

matches = re.findall(answer_key_regex, all_text, re.DOTALL)

i=1
for match in matches:
    answer = match[0]
    explanation = match[1].strip()
    answer_dict[str(i)]=answer
    i+=1
answer_dict

{'1': 'A',
 '2': 'B',
 '3': 'B',
 '4': 'B',
 '5': 'C',
 '6': 'C',
 '7': 'A',
 '8': 'C',
 '9': 'B',
 '10': 'B',
 '11': 'C',
 '12': 'B',
 '13': 'A',
 '14': 'A',
 '15': 'B',
 '16': 'C',
 '17': 'A',
 '18': 'D',
 '19': 'B',
 '20': 'D',
 '21': 'C',
 '22': 'A',
 '23': 'D',
 '24': 'A',
 '25': 'B'}

In [5]:
explanation_dict={}
explanation_regex = r"☑\s*[ABCD]\.\s*(.*?)\n(?:☐✗\n(.*?\n)|$)"

matches = re.findall(explanation_regex, all_text, re.DOTALL)
i=1
for match in matches:
    explanation_correct = match[0].strip()
    explanation_incorrect = match[1].strip() if match[1] else ""
    explanation_dict[str(i)]=f"{explanation_correct} {explanation_incorrect}"
    i+=1
explanation_dict

{'1': 'These are internal enterprise environmental factors, because the factors are internal to\nthe organization, but they do restrict options for the project. B, C, and D are incorrect. B is incorrect because external enterprise environmental factors',
 '2': 'Laws and regulations are external enterprise environmental factors. A, C, and D are incorrect. A, constraints, are anything that limit your options, such as',
 '3': 'This is an example of a matrix structure. Although the question doesn’t define whether\nthe matrix is truly strong, weak, or balanced, weak matrix is the best answer presented. A, C, and D are incorrect. A is incorrect because a virtual team describes an organizational',
 '4': 'Communication is the key general management skill a project manager will use the most. A, C, and D are incorrect. They are necessary, but communication accounts for the majority',
 '5': 'Managing has to do with consistently producing key results that are expected by stakeholders. A, B, and D 

In [6]:
import re
import unicodedata  # Module Unicode

matchesS = re.finditer(section, all_text, re.MULTILINE)
pmpqa = []

for sectionNum, sectionMatch in enumerate(matchesS):
    matchesQN = re.findall(qnumber, sectionMatch.group(0), re.MULTILINE)
    matchesQ = re.findall(question, sectionMatch.group(0), re.MULTILINE)
    matchesA = re.findall(choiceA, sectionMatch.group(0), re.MULTILINE)
    matchesB = re.findall(choiceB, sectionMatch.group(0), re.MULTILINE)
    matchesC = re.findall(choiceC, sectionMatch.group(0), re.MULTILINE)
    matchesD = re.findall(choiceD, sectionMatch.group(0), re.MULTILINE)
    
    for matchQN, matchQ, matchA, matchB, matchC, matchD in zip(matchesQN, matchesQ, matchesA, matchesB, matchesC, matchesD):
        # Normalize Unicode characters to their closest ASCII representation
        # This removes accents and converts special characters like '\u2018' and '\u2019' to standard ASCII equivalents.
        matchQ = unicodedata.normalize('NFKD', matchQ).encode('ascii', 'ignore').decode('utf-8')
        matchA = unicodedata.normalize('NFKD', matchA).encode('ascii', 'ignore').decode('utf-8')
        matchB = unicodedata.normalize('NFKD', matchB).encode('ascii', 'ignore').decode('utf-8')
        matchC = unicodedata.normalize('NFKD', matchC).encode('ascii', 'ignore').decode('utf-8')
        matchD = unicodedata.normalize('NFKD', matchD).encode('ascii', 'ignore').decode('utf-8')
        
        qadata = {}
        qadata['source'] = 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education'
        qadata['year'] = 2018
        qadata['type'] = 'study guide'
        qadata['qid'] = matchQN
        qadata['question'] = {"stem": matchQ.replace('\n',' ').strip(),
                              "choices": [
                                  {"label": "A", "text": matchA.replace('\n',' ').strip()},
                                  {"label": "B", "text": matchB.replace('\n',' ').strip()},
                                  {"label": "C", "text": matchC.replace('\n',' ').strip()},
                                  {"label": "D", "text": matchD.replace('\n',' ').strip()}]}
    
        
        #if matchQN in answer_dict.keys():
        qadata['answerKey'] = answer_dict.get(matchQN)
        
        statements = []
        for choice, text in zip(['A', 'B', 'C', 'D'], [matchA, matchB, matchC, matchD]):
            is_correct = (answer_dict.get(matchQN) == choice)
            statements.append({
                "label": is_correct,
                "statement": matchQ.replace('\n',' ').strip() + ' ' + text.replace('\n', ' ').strip()
            })
        exp = unicodedata.normalize('NFKD', explanation_dict.get(matchQN)).encode('ascii', 'ignore').decode('utf-8')
        qadata['explanation'] = exp
        qadata['statements'] = statements
        pmpqa.append(qadata)

pmpqa


[{'source': 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education',
  'year': 2018,
  'type': 'study guide',
  'qid': '1',
  'question': {'stem': 'You are the project manager for the application deployment project. This project will deploy software to 450 sites throughout North America and in Europe. Because this project will span multiple countries, you must consider the limitations imposed by communications, accessing resources, and other concerns. These international concerns that limit your choices in the project are known as which of the following?',
   'choices': [{'label': 'A',
     'text': 'Internal enterprise environmental factors'},
    {'label': 'B', 'text': 'External enterprise environmental factors'},
    {'label': 'C', 'text': 'Cultural norms'},
    {'label': 'D', 'text': 'Organizational process assets'}]},
  'answerKey': 'A',
  'explanation': 'These are internal enterprise environmental factors, because the factors are internal to\nthe 

In [7]:
# save the PMP-QA dataset as jsonl
with open(output_file, 'w') as fout:
    for dic in pmpqa:
        fout.write(json.dumps(dic) + '\n')

In [1]:
# import libraries
import re
import pdfplumber
import json

In [2]:
# define PDF file path
file_name = 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education (2018)'
input_file = file_name + '.pdf'
output_file = file_name + '_chapter3_statements.jsonl'

In [3]:
pages_to_extract = list(range(169, 179))

    
all_text = ""

with pdfplumber.open(input_file) as pdf:
    for page_num, page in enumerate(pdf.pages):
        if page_num + 1 in pages_to_extract:
            page_text = page.extract_text()
            if page_text:
                all_text += '\n' + page_text
pattern1 = r"CertPrs_2015/PMP® Project Management Professional Study Guide/Joseph Phillips/86198-8/Chapter \d+"
all_text=re.sub(pattern1, '', all_text)
pattern2 = r"Self Test Answers \d+"
all_text=re.sub(pattern2, '', all_text)
pattern3 = r"SELF TEST ANSWERS"
all_text=re.sub(pattern3, '', all_text)
pattern4 = r"\d+ Chapter 3 Serving as a Project Manager"
all_text=re.sub(pattern4, '', all_text)
pattern5 = r"03-ch03.indd \d+ 28/11/17 3:08 PM"
all_text=re.sub(pattern5, '', all_text)   
#print(all_text)

In [4]:
section = r"^\d+\.\s+([\S\s]*?)(?=\Z|^\d+\.\s+)"
# capture sections from question to question
sections = re.finditer(section, all_text, re.MULTILINE)

qnumber = r"(^\d+)\.\s+[\S\s]"
qnumbers = re.finditer(qnumber, all_text, re.MULTILINE)

question = r"\d+\.\s+([\S\s]*?)\nA\."
questions = re.finditer(question, all_text, re.MULTILINE)

choiceA = r"\nA\.([\S\s]*?)\nB\."
choiceAs = re.finditer(choiceA, all_text, re.MULTILINE)

choiceB = r"\nB\.([\S\s]*?)\nC\."
choiceBs = re.finditer(choiceB, all_text, re.MULTILINE)

choiceC = r"\nC\.([\S\s]*?)\nD\."
choiceCs = re.finditer(choiceC, all_text, re.MULTILINE)

choiceD = r"\nD\.([\S\s]*?)\n"
choiceDs = re.finditer(choiceD, all_text, re.MULTILINE)

answer_dict={}
    
answer_key_regex = r"☑\s*([ABCD])\.\s*(.*?)\n"

matches = re.findall(answer_key_regex, all_text, re.DOTALL)

i=1
for match in matches:
    answer = match[0]
    explanation = match[1].strip()
    answer_dict[str(i)]=answer
    i+=1
answer_dict

{'1': 'A',
 '2': 'B',
 '3': 'B',
 '4': 'B',
 '5': 'C',
 '6': 'C',
 '7': 'A',
 '8': 'C',
 '9': 'B',
 '10': 'B',
 '11': 'C',
 '12': 'B',
 '13': 'A',
 '14': 'A',
 '15': 'B',
 '16': 'C',
 '17': 'A',
 '18': 'D',
 '19': 'B',
 '20': 'D',
 '21': 'C',
 '22': 'A',
 '23': 'D',
 '24': 'A',
 '25': 'B'}

In [5]:
explanation_dict={}
explanation_regex = r"☑\s*[ABCD]\.\s*(.*?)\n(?:☐✗\n(.*?\n)|$)"

matches = re.findall(explanation_regex, all_text, re.DOTALL)
i=1
for match in matches:
    explanation_correct = match[0].strip()
    explanation_incorrect = match[1].strip() if match[1] else ""
    explanation_dict[str(i)]=f"{explanation_correct} {explanation_incorrect}"
    i+=1
explanation_dict

{'1': 'Active listening is the participatory component of a conversation that confirms what was\nsaid and enables the sender to offer clarity, if needed. B, C, and D are incorrect. B is incorrect because the sender–receiver model is a model',
 '2': 'The purpose of negotiations is to reach a fair agreement for all parties involved. A, C, and D are incorrect. A and C are incorrect because these two choices are mutually',
 '3': 'Communication skills and a positive attitude are most helpful in influencing an organization. A, C, and D are incorrect. A is incorrect because management and leadership are values for',
 '4': 'Communication is the key general management skill a project manager will use the most. A, C, and D are incorrect. These choices are important to project management, but',
 '5': 'Sammy is consciously incompetent because she is aware that she needs more training to\nbe competent in a new skill. A, B, and D are incorrect. A is incorrect because unconsciously competent project 

In [6]:
import re
import unicodedata  # Module Unicode

matchesS = re.finditer(section, all_text, re.MULTILINE)
pmpqa = []

for sectionNum, sectionMatch in enumerate(matchesS):
    matchesQN = re.findall(qnumber, sectionMatch.group(0), re.MULTILINE)
    matchesQ = re.findall(question, sectionMatch.group(0), re.MULTILINE)
    matchesA = re.findall(choiceA, sectionMatch.group(0), re.MULTILINE)
    matchesB = re.findall(choiceB, sectionMatch.group(0), re.MULTILINE)
    matchesC = re.findall(choiceC, sectionMatch.group(0), re.MULTILINE)
    matchesD = re.findall(choiceD, sectionMatch.group(0), re.MULTILINE)
    
    for matchQN, matchQ, matchA, matchB, matchC, matchD in zip(matchesQN, matchesQ, matchesA, matchesB, matchesC, matchesD):
        # Normalize Unicode characters to their closest ASCII representation
        # This removes accents and converts special characters like '\u2018' and '\u2019' to standard ASCII equivalents.
        matchQ = unicodedata.normalize('NFKD', matchQ).encode('ascii', 'ignore').decode('utf-8')
        matchA = unicodedata.normalize('NFKD', matchA).encode('ascii', 'ignore').decode('utf-8')
        matchB = unicodedata.normalize('NFKD', matchB).encode('ascii', 'ignore').decode('utf-8')
        matchC = unicodedata.normalize('NFKD', matchC).encode('ascii', 'ignore').decode('utf-8')
        matchD = unicodedata.normalize('NFKD', matchD).encode('ascii', 'ignore').decode('utf-8')
        
        qadata = {}
        qadata['source'] = 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education'
        qadata['year'] = 2018
        qadata['type'] = 'study guide'
        qadata['qid'] = matchQN
        qadata['question'] = {"stem": matchQ.replace('\n',' ').strip(),
                              "choices": [
                                  {"label": "A", "text": matchA.replace('\n',' ').strip()},
                                  {"label": "B", "text": matchB.replace('\n',' ').strip()},
                                  {"label": "C", "text": matchC.replace('\n',' ').strip()},
                                  {"label": "D", "text": matchD.replace('\n',' ').strip()}]}
    
        
        #if matchQN in answer_dict.keys():
        qadata['answerKey'] = answer_dict.get(matchQN)
        
        statements = []
        for choice, text in zip(['A', 'B', 'C', 'D'], [matchA, matchB, matchC, matchD]):
            is_correct = (answer_dict.get(matchQN) == choice)
            statements.append({
                "label": is_correct,
                "statement": matchQ.replace('\n',' ').strip() + ' ' + text.replace('\n', ' ').strip()
            })
        exp = unicodedata.normalize('NFKD', explanation_dict.get(matchQN)).encode('ascii', 'ignore').decode('utf-8')
        qadata['explanation'] = exp
        qadata['statements'] = statements
        pmpqa.append(qadata)

pmpqa


[{'source': 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education',
  'year': 2018,
  'type': 'study guide',
  'qid': '1',
  'question': {'stem': 'Leadership and project management are connected. As a leader and a manager, you will rely on communications within a project to help motivate, manage, and ensure that the project is moving forward toward its objectives. In one component of communication, the receiver restates what the sender has said to clarify the message and to give the sender an opportunity to offer clarity if needed. What is this communication component called?',
   'choices': [{'label': 'A', 'text': 'Active listening'},
    {'label': 'B', 'text': 'Senderreceiver model'},
    {'label': 'C', 'text': 'Communications planning'},
    {'label': 'D', 'text': 'Leader listening'}]},
  'answerKey': 'A',
  'explanation': 'Active listening is the participatory component of a conversation that confirms what was\nsaid and enables the sender to offer

In [7]:
# save the PMP-QA dataset as jsonl
with open(output_file, 'w') as fout:
    for dic in pmpqa:
        fout.write(json.dumps(dic) + '\n')

In [1]:
# import libraries
import re
import pdfplumber
import json

In [2]:
# define PDF file path
file_name = 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education (2018)'
input_file = file_name + '.pdf'
output_file = file_name + '_chapter4_statements.jsonl'

In [3]:
pages_to_extract = list(range(249, 259))

    
all_text = ""

with pdfplumber.open(input_file) as pdf:
    for page_num, page in enumerate(pdf.pages):
        if page_num + 1 in pages_to_extract:
            page_text = page.extract_text()
            if page_text:
                all_text += '\n' + page_text
pattern1 = r"CertPrs_2015/PMP® Project Management Professional Study Guide/Joseph Phillips/86198-8/Chapter \d+"
all_text=re.sub(pattern1, '', all_text)
pattern2 = r"Self Test Answers \d+"
all_text=re.sub(pattern2, '', all_text)
pattern3 = r"SELF TEST ANSWERS"
all_text=re.sub(pattern3, '', all_text)
pattern4 = r"\d+ Chapter 4 Implementing Project Integration Management"
all_text=re.sub(pattern4, '', all_text)
pattern5 = r"04-ch04.indd \d+ 06/11/17 1:47 PM"
all_text=re.sub(pattern5, '', all_text)   
#print(all_text)

In [4]:
section = r"^\d+\.\s+([\S\s]*?)(?=\Z|^\d+\.\s+)"
# capture sections from question to question
sections = re.finditer(section, all_text, re.MULTILINE)

qnumber = r"(^\d+)\.\s+[\S\s]"
qnumbers = re.finditer(qnumber, all_text, re.MULTILINE)

question = r"\d+\.\s+([\S\s]*?)\nA\."
questions = re.finditer(question, all_text, re.MULTILINE)

choiceA = r"\nA\.([\S\s]*?)\nB\."
choiceAs = re.finditer(choiceA, all_text, re.MULTILINE)

choiceB = r"\nB\.([\S\s]*?)\nC\."
choiceBs = re.finditer(choiceB, all_text, re.MULTILINE)

choiceC = r"\nC\.([\S\s]*?)\nD\."
choiceCs = re.finditer(choiceC, all_text, re.MULTILINE)

choiceD = r"\nD\.([\S\s]*?)\n"
choiceDs = re.finditer(choiceD, all_text, re.MULTILINE)

answer_dict={}
    
answer_key_regex = r"☑\s*([ABCD])\.\s*(.*?)\n"

matches = re.findall(answer_key_regex, all_text, re.DOTALL)

i=1
for match in matches:
    answer = match[0]
    explanation = match[1].strip()
    answer_dict[str(i)]=answer
    i+=1
answer_dict

{'1': 'A',
 '2': 'C',
 '3': 'D',
 '4': 'B',
 '5': 'A',
 '6': 'B',
 '7': 'A',
 '8': 'D',
 '9': 'A',
 '10': 'D',
 '11': 'A',
 '12': 'B',
 '13': 'D',
 '14': 'C',
 '15': 'A',
 '16': 'B',
 '17': 'A',
 '18': 'D',
 '19': 'C',
 '20': 'B',
 '21': 'B',
 '22': 'C',
 '23': 'C',
 '24': 'D',
 '25': 'A'}

In [5]:
explanation_dict={}
explanation_regex = r"☑\s*[ABCD]\.\s*(.*?)\n(?:☐✗\n(.*?\n)|$)"

matches = re.findall(explanation_regex, all_text, re.DOTALL)
i=1
for match in matches:
    explanation_correct = match[0].strip()
    explanation_incorrect = match[1].strip() if match[1] else ""
    explanation_dict[str(i)]=f"{explanation_correct} {explanation_incorrect}"
    i+=1
explanation_dict

{'1': 'Projects are selected based on business needs first. B, C, and D are incorrect. B is incorrect because project constraints are typically not an issue',
 '2': 'The project team contributes to the lessons learned document. The project manager also\ncontributes, or leads, the creation, but this is not a choice in the question. A, B, and D are incorrect. A is incorrect because the customers do not always contribute',
 '3': 'Additional quality testing will require additional time and resources for the project. This is\nan example of integrated change control. A, B, and C are incorrect. A is incorrect because scope creep includes small, undocumented',
 '4': 'Change requests do not always require more money. Approved changes may require more\nfunds, but not always. The change request may be denied, so no additional funds are needed for\nthe project. A, C, and D are incorrect. These are characteristics of change requests that occur during a',
 '5': 'A formal, documented change request i

In [6]:
import re
import unicodedata  # Module Unicode

matchesS = re.finditer(section, all_text, re.MULTILINE)
pmpqa = []

for sectionNum, sectionMatch in enumerate(matchesS):
    matchesQN = re.findall(qnumber, sectionMatch.group(0), re.MULTILINE)
    matchesQ = re.findall(question, sectionMatch.group(0), re.MULTILINE)
    matchesA = re.findall(choiceA, sectionMatch.group(0), re.MULTILINE)
    matchesB = re.findall(choiceB, sectionMatch.group(0), re.MULTILINE)
    matchesC = re.findall(choiceC, sectionMatch.group(0), re.MULTILINE)
    matchesD = re.findall(choiceD, sectionMatch.group(0), re.MULTILINE)
    
    for matchQN, matchQ, matchA, matchB, matchC, matchD in zip(matchesQN, matchesQ, matchesA, matchesB, matchesC, matchesD):
        # Normalize Unicode characters to their closest ASCII representation
        # This removes accents and converts special characters like '\u2018' and '\u2019' to standard ASCII equivalents.
        matchQ = unicodedata.normalize('NFKD', matchQ).encode('ascii', 'ignore').decode('utf-8')
        matchA = unicodedata.normalize('NFKD', matchA).encode('ascii', 'ignore').decode('utf-8')
        matchB = unicodedata.normalize('NFKD', matchB).encode('ascii', 'ignore').decode('utf-8')
        matchC = unicodedata.normalize('NFKD', matchC).encode('ascii', 'ignore').decode('utf-8')
        matchD = unicodedata.normalize('NFKD', matchD).encode('ascii', 'ignore').decode('utf-8')
        
        qadata = {}
        qadata['source'] = 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education'
        qadata['year'] = 2018
        qadata['type'] = 'study guide'
        qadata['qid'] = matchQN
        qadata['question'] = {"stem": matchQ.replace('\n',' ').strip(),
                              "choices": [
                                  {"label": "A", "text": matchA.replace('\n',' ').strip()},
                                  {"label": "B", "text": matchB.replace('\n',' ').strip()},
                                  {"label": "C", "text": matchC.replace('\n',' ').strip()},
                                  {"label": "D", "text": matchD.replace('\n',' ').strip()}]}
    
        
        #if matchQN in answer_dict.keys():
        qadata['answerKey'] = answer_dict.get(matchQN)
        
        statements = []
        for choice, text in zip(['A', 'B', 'C', 'D'], [matchA, matchB, matchC, matchD]):
            is_correct = (answer_dict.get(matchQN) == choice)
            statements.append({
                "label": is_correct,
                "statement": matchQ.replace('\n',' ').strip() + ' ' + text.replace('\n', ' ').strip()
            })
        exp = unicodedata.normalize('NFKD', explanation_dict.get(matchQN)).encode('ascii', 'ignore').decode('utf-8')
        qadata['explanation'] = exp
        qadata['statements'] = statements
        pmpqa.append(qadata)

pmpqa


[{'source': 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education',
  'year': 2018,
  'type': 'study guide',
  'qid': '1',
  'question': {'stem': 'You are a project manager for your organization. Management has asked you to help determine which projects should be selected for implementation. In a project selection model, which of the following is the most important factor?',
   'choices': [{'label': 'A', 'text': 'Business needs'},
    {'label': 'B', 'text': 'Type of constraints'},
    {'label': 'C', 'text': 'Budget'},
    {'label': 'D', 'text': 'Schedule'}]},
  'answerKey': 'A',
  'explanation': 'Projects are selected based on business needs first. B, C, and D are incorrect. B is incorrect because project constraints are typically not an issue',
  'statements': [{'label': True,
    'statement': 'You are a project manager for your organization. Management has asked you to help determine which projects should be selected for implementation. In a project

In [7]:
# save the PMP-QA dataset as jsonl
with open(output_file, 'w') as fout:
    for dic in pmpqa:
        fout.write(json.dumps(dic) + '\n')

In [1]:
# import libraries
import re
import pdfplumber
import json

In [2]:
# define PDF file path
file_name = 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education (2018)'
input_file = file_name + '.pdf'
output_file = file_name + '_chapter5_statements.jsonl'

In [3]:
pages_to_extract = list(range(303, 313))

    
all_text = ""

with pdfplumber.open(input_file) as pdf:
    for page_num, page in enumerate(pdf.pages):
        if page_num + 1 in pages_to_extract:
            page_text = page.extract_text()
            if page_text:
                all_text += '\n' + page_text
pattern1 = r"CertPrs_2015/PMP® Project Management Professional Study Guide/Joseph Phillips/86198-8/Chapter \d+"
all_text=re.sub(pattern1, '', all_text)
pattern2 = r"Self Test Answers \d+"
all_text=re.sub(pattern2, '', all_text)
pattern3 = r"SELF TEST ANSWERS"
all_text=re.sub(pattern3, '', all_text)
pattern4 = r"\d+ Chapter 5 Managing the Project Scope"
all_text=re.sub(pattern4, '', all_text)
pattern5 = r"05-ch05.indd \d+ 06/11/17 1:50 PM"
all_text=re.sub(pattern5, '', all_text)   
#print(all_text)

In [4]:
section = r"^\d+\.\s+([\S\s]*?)(?=\Z|^\d+\.\s+)"
# capture sections from question to question
sections = re.finditer(section, all_text, re.MULTILINE)

qnumber = r"(^\d+)\.\s+[\S\s]"
qnumbers = re.finditer(qnumber, all_text, re.MULTILINE)

question = r"\d+\.\s+([\S\s]*?)\nA\."
questions = re.finditer(question, all_text, re.MULTILINE)

choiceA = r"\nA\.([\S\s]*?)\nB\."
choiceAs = re.finditer(choiceA, all_text, re.MULTILINE)

choiceB = r"\nB\.([\S\s]*?)\nC\."
choiceBs = re.finditer(choiceB, all_text, re.MULTILINE)

choiceC = r"\nC\.([\S\s]*?)\nD\."
choiceCs = re.finditer(choiceC, all_text, re.MULTILINE)

choiceD = r"\nD\.([\S\s]*?)\n"
choiceDs = re.finditer(choiceD, all_text, re.MULTILINE)

answer_dict={}
    
answer_key_regex = r"☑\s*([ABCD])\.\s*(.*?)\n"

matches = re.findall(answer_key_regex, all_text, re.DOTALL)

i=1
for match in matches:
    answer = match[0]
    explanation = match[1].strip()
    answer_dict[str(i)]=answer
    i+=1
answer_dict

{'1': 'B',
 '2': 'C',
 '3': 'C',
 '4': 'B',
 '5': 'C',
 '6': 'C',
 '7': 'B',
 '8': 'C',
 '9': 'A',
 '10': 'B',
 '11': 'B',
 '12': 'C',
 '13': 'C',
 '14': 'C',
 '15': 'B',
 '16': 'B',
 '17': 'B',
 '18': 'C',
 '19': 'B',
 '20': 'B',
 '21': 'D',
 '22': 'C',
 '23': 'C',
 '24': 'A',
 '25': 'B'}

In [5]:
explanation_dict={}
explanation_regex = r"☑\s*[ABCD]\.\s*(.*?)\n(?:☐✗\n(.*?\n)|$)"

matches = re.findall(explanation_regex, all_text, re.DOTALL)
i=1
for match in matches:
    explanation_correct = match[0].strip()
    explanation_incorrect = match[1].strip() if match[1] else ""
    explanation_dict[str(i)]=f"{explanation_correct} {explanation_incorrect}"
    i+=1
explanation_dict

{'1': 'This is an example of the nominal group technique, in which you ask for as many ideas\nand solutions as possible, and then the group ranks the concepts to help guide the requirements\ndevelopment. A, C, and D are incorrect. A is incorrect because brainstorming is similar to this concept, but',
 '2': 'The WBS is not an organizational process asset. A, B, and D are incorrect. These responses are examples of organizational process assets.',
 '3': 'Scope management plans can be based on templates. For the record, so can the WBS and\nproject scope change control forms. A, B, and D are incorrect. These documents do not stem from templates.',
 '4': 'This is an example of an assumption, since the workstations must meet the hardware\nrequirements. A, C, and D are incorrect. A and C are incorrect because the scenario did not describe a risk',
 '5': 'A project that must be completed by a deadline is dealing with schedule constraints. A, B, and D are incorrect. A is incorrect because the co

In [6]:
import re
import unicodedata  # Module Unicode

matchesS = re.finditer(section, all_text, re.MULTILINE)
pmpqa = []

for sectionNum, sectionMatch in enumerate(matchesS):
    matchesQN = re.findall(qnumber, sectionMatch.group(0), re.MULTILINE)
    matchesQ = re.findall(question, sectionMatch.group(0), re.MULTILINE)
    matchesA = re.findall(choiceA, sectionMatch.group(0), re.MULTILINE)
    matchesB = re.findall(choiceB, sectionMatch.group(0), re.MULTILINE)
    matchesC = re.findall(choiceC, sectionMatch.group(0), re.MULTILINE)
    matchesD = re.findall(choiceD, sectionMatch.group(0), re.MULTILINE)
    
    for matchQN, matchQ, matchA, matchB, matchC, matchD in zip(matchesQN, matchesQ, matchesA, matchesB, matchesC, matchesD):
        # Normalize Unicode characters to their closest ASCII representation
        # This removes accents and converts special characters like '\u2018' and '\u2019' to standard ASCII equivalents.
        matchQ = unicodedata.normalize('NFKD', matchQ).encode('ascii', 'ignore').decode('utf-8')
        matchA = unicodedata.normalize('NFKD', matchA).encode('ascii', 'ignore').decode('utf-8')
        matchB = unicodedata.normalize('NFKD', matchB).encode('ascii', 'ignore').decode('utf-8')
        matchC = unicodedata.normalize('NFKD', matchC).encode('ascii', 'ignore').decode('utf-8')
        matchD = unicodedata.normalize('NFKD', matchD).encode('ascii', 'ignore').decode('utf-8')
        
        qadata = {}
        qadata['source'] = 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education'
        qadata['year'] = 2018
        qadata['type'] = 'study guide'
        qadata['qid'] = matchQN
        qadata['question'] = {"stem": matchQ.replace('\n',' ').strip(),
                              "choices": [
                                  {"label": "A", "text": matchA.replace('\n',' ').strip()},
                                  {"label": "B", "text": matchB.replace('\n',' ').strip()},
                                  {"label": "C", "text": matchC.replace('\n',' ').strip()},
                                  {"label": "D", "text": matchD.replace('\n',' ').strip()}]}
    
        
        #if matchQN in answer_dict.keys():
        qadata['answerKey'] = answer_dict.get(matchQN)
        
        statements = []
        for choice, text in zip(['A', 'B', 'C', 'D'], [matchA, matchB, matchC, matchD]):
            is_correct = (answer_dict.get(matchQN) == choice)
            statements.append({
                "label": is_correct,
                "statement": matchQ.replace('\n',' ').strip() + ' ' + text.replace('\n', ' ').strip()
            })
        exp = unicodedata.normalize('NFKD', explanation_dict.get(matchQN)).encode('ascii', 'ignore').decode('utf-8')
        qadata['explanation'] = exp
        qadata['statements'] = statements
        pmpqa.append(qadata)

pmpqa


[{'source': 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education',
  'year': 2018,
  'type': 'study guide',
  'qid': '1',
  'question': {'stem': 'You are the project manager of the OQH Project and are working with the project stakeholders to determine the project requirements. You and the stakeholders are brainstorming privately to come up with as many solutions to the project as possible. After the brainstorming session, a recorder documents all the solutions on a white board so everyone can see the ideas and how they may be related. After the solutions have been documented, you lead the group through a voting process to discuss and rank each idea and requirement that has been proposed. What is this requirement gathering technique called?',
   'choices': [{'label': 'A', 'text': 'Brainstorming'},
    {'label': 'B', 'text': 'Nominal group technique'},
    {'label': 'C', 'text': 'Affinity diagram creations'},
    {'label': 'D', 'text': 'Mind mapping'}]

In [7]:
# save the PMP-QA dataset as jsonl
with open(output_file, 'w') as fout:
    for dic in pmpqa:
        fout.write(json.dumps(dic) + '\n')

In [1]:
# import libraries
import re
import pdfplumber
import json

In [2]:
# define PDF file path
file_name = 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education (2018)'
input_file = file_name + '.pdf'
output_file = file_name + '_chapter6_statements.jsonl'

In [3]:
pages_to_extract = list(range(377, 389))

    
all_text = ""

with pdfplumber.open(input_file) as pdf:
    for page_num, page in enumerate(pdf.pages):
        if page_num + 1 in pages_to_extract:
            page_text = page.extract_text()
            if page_text:
                all_text += '\n' + page_text
pattern1 = r"CertPrs_2015/PMP® Project Management Professional Study Guide/Joseph Phillips/86198-8/Chapter \d+"
all_text=re.sub(pattern1, '', all_text)
pattern2 = r"Self Test Answers \d+"
all_text=re.sub(pattern2, '', all_text)
pattern3 = r"SELF TEST ANSWERS"
all_text=re.sub(pattern3, '', all_text)
pattern4 = r"\d+ Chapter 6 Introducing Project Schedule Management"
all_text=re.sub(pattern4, '', all_text)
pattern5 = r"06-ch06.indd \d+ 28/11/17 3:09 PM"
all_text=re.sub(pattern5, '', all_text)   
all_text='\n1. You are the project manager of the JHG Project. This project has 32 stakeholders and will require\nimplementation activities in North and South America. You have been requested to provide a\nduration estimate for the project. Of the following, which will offer the best level of detail in your\nestimate?\nA. The resource calendar\nB. An order of magnitude\nC. A requirements document\nD. A stakeholder analysis\n☑ A. The resource calendar is the best choice for this scenario because it is the only activity\nduration estimating input listed.\n☐✗\nB, C, and D are incorrect. B is incorrect because the order of magnitude provides little\ninformation for accurate estimating. C, while tempting, is incorrect because the requirements\ndocument lists the high-level deliverables, while the WBS provides more detail. D is incorrect\nbecause stakeholder analysis does not provide enough information to predict accurately when the\nproject will end.\n\n\n\n2. Michael is the project manager of the 78GH Project. This project requires several members of the\nproject team to complete a certification class for another project the week of November 2. This\nclass causes some of the project activities on Michael’s activities to be delayed from his target\nschedule. This is an example of which of the following?\nA. Hard logic\nB. External dependencies\nC. Soft logic\nD. Conflict of interest\n☑ B. Before the work can begin, the certification class external to the work must be completed.\n☐✗\nA, C, and D are incorrect. A is incorrect; hard logic is the mandatory sequencing of events.\nC is incorrect because there is no preferential (soft) logic. D is incorrect because it does not apply\nto this scenario.\n3. You are managing an interior decorating project. The walls are scheduled to be painted immediately\nafter the primer is applied. You have allowed 36 hours between the primer activity and the painting\nactivity to ensure that the primer has cured. This is an example of which one of the following?\nA. Lead\nB. Lag\nC. Soft logic\nD. Finish-to-start relationship\n☑ B. The time between the activities is lag time. The painting activity must wait 36 hours\nbefore it can begin.\n☐✗\nA, C, and D are incorrect. A, lead time, is when the activities are brought closer together\nor even overlap. C, soft logic, describes when activities are scheduled based on preferences,\nguidelines, or external conditions. D is incorrect, because, although this does describe a finish-\nto-start relationship, lag is a better choice because of the added waiting time.\n\n\n\n4. You are the project manager of the DFK Project and you’re reviewing your project network diagram.\nYou are considering changing the relationship of some of the project activities to reduce the duration\nof the project work. Consider your project’s network diagram (as shown in the following illustration).\nES=12 EF=13\n2\nES=1 3 EF=3 ES=4 5 EF=8 C ES=14 EF=20\n7\nA B\nD\nLS=14 LF=20\nStart\nEnd\nES=1 9 EF=9\nE ES=10 2 EF=11 ES=12 4 EF=15\nF G\nA. FS\nB. SS\nC. FF\nD. SF\n☑ A. G is slated to start immediately after F, so this is a finish-to-start (FS) relationship. In other\nwords, F must finish so G may start.\n☐✗\nB, C, and D are incorrect. These relationships do not fit the scenario.\n5. You are the project manager for the LLL Project. Steven, a project team member, is confused\nabout network diagrams. Specifically, he wants to know what the critical path is in a network\ndiagram. Your answer is which one of the following?\nA. The critical path is the network that hosts the activities most critical to the project’s success.\nB. The critical path is a path with the longest duration.\nC. The critical path is always one path that cannot be delayed, or the entire project will be delayed.\nD. The critical path is the path from start to completion with no deviation from the project plan.\n☑ B. The critical path is always the path with the longest duration; it’s also the minimum\namount of time needed to complete the project.\n☐✗\nA, C, and D are incorrect. A is incorrect because the critical path hosts the activities, not a\nnetwork. C is a distraction answer and is incorrect because there can be more than one critical\npath in a network diagram. D is incorrect because it does not adequately describe the critical path.\n\n\n\n6. You are the project manager of the HQQ Project and you’re working with your customer stakeholder.\nThe stakeholder has asked that you find a method to reduce the overall project duration for her\norganization. She has promised a bonus to your company of $10,000 per day that you finish ahead of\nschedule. Which duration compression technique could you utilize on activities that are effort-driven?\nA. Crashing\nB. Fast tracking\nC. Effort-driven activities cannot be compressed\nD. Resource smoothing\n☑ A. Crashing enables you to add more labor to effort-driven activities. By adding more labor,\nor effort, you are reducing the duration of the activity.\n☐✗\nB, C, and D are incorrect. B is incorrect because fast tracking enables you to overlap phases\nin your project and isn’t the best choice for the question. C is incorrect because effort-driven\nactivities are ideal activities to be reduced through additional labor. D is incorrect because resource\nsmoothing refers to the leveling of resources on noncritical path activities. This approach doesn’t\nreduce the duration of activities.\n7. You are creating a schedule duration estimate for the activities in the PDR Project. You’re working\nwith your project team and comparing the results of a past similar project to predict the schedule\nfor the current project. What estimating approach are you using?\nA. Organizational process assets\nB. Parametric\nC. Analogous\nD. PERT\n☑ C. This is an example of an analogous estimate. You can remember this estimate approach by\nidentifying the “analogy” between the two similar projects.\n☐✗\nA, B, and D are incorrect. A is incorrect because, although the previous project data is part\nof organizational process assets, the assets are not an estimating approach. B is incorrect because\nparametric estimates use a parameter, such as five hours per unit. D is also incorrect because\nPERT uses an average of the optimistic, most likely, and pessimistic time estimates for each activity.\n8. You are the project manager for your organization, which utilizes an agile approach to project\nmanagement. The development team is rating the difficulty of completing the requirements in the\ncurrent sprint. What term describes rating of the requirements to determine how many requirements\ncan be completed in a sprint?\nA. Fist-to-five voting\nB. Value analysis\nC. Sprint backlog\nD. Story points\n\n\n\n☑ D. Requirements are written as user stories. Story points score the difficulty of the tasks\nneeded to complete a user story in a sprint.\n☐✗\nA, B, and C are incorrect. These terms are not processes that rate requirements to determine\nhow many requirements can be completed in a sprint.\n9. You are the project manager for the POL Project. This project will use a three-point estimate to\ncalculate the estimates for activity duration. For Activity D, you have the following information:\nP = 9, O = 4, M = 5. What is the result of this estimate?\nA. 18 weeks\nB. 6 weeks\nC. 33.33 days\nD. 3 weeks\n☑ B. The formula is (P + M + O)/3. In this instance, the outcome is 6 weeks.\n☐✗\nA, C, and D are incorrect. These calculations are incorrect.\n10. You are the project manager for the YKL Project. This project will affect several lines of business\nat completion. You have elected to schedule each milestone in the project to end so the work does\nnot affect current business cycles. This is an example of which one of the following?\nA. Constraint\nB. Expert judgment\nC. WBS scheduling\nD. Soft logic\n☑ D. Soft logic enables the project manager to make decisions based on conditions outside of\nthe project, best practices, or guidelines.\n☐✗\nA, B, and C are incorrect. A is incorrect because this is not an example of constraints, since\nthe project manager is not required to use soft logic. B and C are incorrect; they do not describe\nthe scenario fully.\n11. You are the project manager for the MNB Project. You and your project team are about to enter the\nactivity duration estimating process. Which of the following will not be helpful in your meeting?\nA. Constraints\nB. Assumptions\nC. The project charter\nD. Identified risks\n☑ C. The project charter is not an input to the activity duration estimating process.\n☐✗\nA, B, and D are incorrect. These are inputs to activity duration estimating.\n\n\n\n12. You are the project manager for a new training program at your customer’s site. This program will\nrequire each of the customer’s employees to attend a half-day class and complete an assessment\nexam. You will be completing the training at the customer’s facility and will need a trainer for the\nduration of the training, which is six months. This is an example of which of the following?\nA. Resource requirements\nB. Assumption\nC. Cost constraint\nD. A human resource issue\n☑ A. The trainer resource is required for the project for six months.\n☐✗\nB, C, and D are incorrect. They do not describe the resource requirement of the trainer on\nthe project.\n13. You are the project manager for a construction company. Your firm has been contracted to\ncomplete the drilling of a well for a new cabin in Arkansas. The specification of the well is\ndocumented, but your company has little experience in well drilling in Arkansas. The stakeholder\nis concerned that your time estimates are not accurate, since the soil and rock in Arkansas are\nquite different from the soil in your home state. Which one of the following can you use to ensure\nyour project estimates are accurate?\nA. An order of magnitude\nB. A commercial duration estimating database\nC. Local contractors\nD. Soil samplings from the Arkansas government\n☑ B. Commercial duration-estimating databases are valid resources to confirm or base time\nestimates on.\n☐✗\nA, C, and D are incorrect. A is incorrect because an order of magnitude offers very little detail\non time estimates. C is incorrect because local contractors are not the best source for confirming\ntime estimates; the question does not define whether the contractors are local to Arkansas or\nto your home state. D is incorrect because commercial duration estimating databases are much\nmore reliable in this scenario.\n14. You are the project manager for your organization. You and your project team are in conflict on\nthe amount of time allotted to complete certain activities. Several of the team members want to\nbloat the time associated with activities to ensure they will have enough time to complete their\ntasks should something go awry. The law of economics that these tasks may suffer from is which\none of the following?\nA. Parkinson’s Law\nB. The law of diminishing returns\nC. Hertzberg’s theory of motivation\nD. Oligopoly\n\n\n\n☑ A. Parkinson’s Law states that work will expand to fulfill the time allotted to it. Bloated tasks\nwill take all the time allotted. Management reserve should be used instead.\n☐✗\nB, C, and D are incorrect. B is incorrect because this describes the relationship between\neffort, duration, and the maximum yield. C is incorrect because Hertzberg’s theory of motivation\ndescribes personalities and worker motivation. D is incorrect because an oligopoly is a procurement\nissue in which there are few vendors available to choose from. Plus, the vendors may seemingly\nhave checks and balances with each other.\n15. You are the project manager for your organization. You and your project team are in conflict\nregarding the amount of time allotted to complete certain activities. Several of the team members\nwant to bloat the time associated with activities to ensure they will have enough time to complete\ntheir tasks should some risk events cause the schedule to change. Instead of overestimating their\nproject activities, the project team should use which of the following?\nA. Capital reserve\nB. Contingency plans\nC. Contingency reserve\nD. Assumptions of plus or minus a percentage\n☑ C. Rather than bloat activities, projects should use a contingency reserve. A contingency\nreserve is a portion of the project schedule allotted for time overruns on activities.\n☐✗\nA, B, and D are incorrect. A is incorrect because it does not describe the scenario. B is incorrect\nbecause contingency plans are a response to risk situations. D is incorrect because it describes a range\nof variance.\n16. You are the project manager for your organization and part of your role requires that you coach the\nproject team on the project management processes. At this point in your project, you’re estimating\nthe activity durations with the project team and need to review the tools and techniques that are\nappropriate for use with this process. Which one of the following is not a tool and technique for the\nestimate activity duration process??\nA. Risk identification\nB. Analogous estimating\nC. Reserve analysis\nD. Three-point estimating\n☑ A. Risk identification is not a tool and technique of the estimate activity durations process.\n☐✗\nB, C, and D are incorrect. These are tools and techniques of activity duration estimating. This\nprocess has six tools to consider: expert judgment, analogous estimating, parametric estimating,\nthree-point estimating, Delphi technique, and reserve analysis.\n\n\n\n17. You are the project manager for the 987 Project. Should this project run over schedule, it will\ncost your organization $35,000 per day in lost sales. With four months to completion, you realize\nthe project is running late. You decide, with management’s approval, to add more project team\nmembers to the plan to complete the work on time. This is an example of which of the following?\nA. Crashing\nB. Fast tracking\nC. Expert judgment\nD. Cost-benefit analysis\n☑ A. When more resources are added to a project to complete the work on time, it is called\ncrashing.\n☐✗\nB, C, and D are incorrect. B is incorrect because fast tracking is the process of changing the\nrelationship between activities to allow tasks to overlap. C is incorrect because expert judgment is\nnot used in this scenario. D is incorrect; cost-benefit analysis may be part of the process to decide\nthe value of adding more workers to the schedule, but it is not the process described.\n18. You are the project manager for the 987 Project. Should this project run over schedule, it will\ncost your organization $35,000 per day in lost sales. With four months to completion, you realize\nthe project is running late. You decide, with management’s approval, to change the relationship\nbetween several of the work packages so they begin in tandem rather than sequentially. This is an\nexample of which one of the following?\nA. Crashing\nB. Fast tracking\nC. Expert judgment\nD. Cost-benefit analysis\n☑ B. Fast tracking allows activities to operate in tandem with each other rather than sequentially.\n☐✗\nA, C, and D are incorrect. A is incorrect; when more resources are added to a project to\ncomplete the work on time, it is called crashing. C is incorrect because expert judgment is not\nused in this scenario. D is incorrect; cost-benefit analysis may be part of the process to decide the\nvalue of fast tracking the schedule, but it is not the process described.\n19. Chris, a project manager for his company, is explaining the difference between a Gantt chart and\na milestone chart. Which of the following best describes a Gantt chart?\nA. A Gantt chart depicts what was planned against what occurred.\nB. A Gantt chart compares the work in the project against the work that has been completed.\nC. A Gantt chart depicts the work in the project against a calendar.\nD. A Gantt chart depicts the work in the project against each resource’s calendar.\n\n\n\n☑ C. A Gantt chart is a bar chart that represents the duration of activities against a calendar.\nThe length of the bars represents the length of activities, while the order of the bars represents the\norder of activities in the project.\n☐✗\nA, B, and D are incorrect. A and B are incorrect because they describe a tracking Gantt. D is\nincorrect because this does not describe a Gantt chart.\n20. Beth is a project manager for her organization. Management has asked Beth to use the CPM\napproach in her network diagram. She is not familiar with this approach and she’s asked you to\nhelp her complete this portion of her project work. Which of the following is a correct attribute\nof the critical path?\nA. It determines the earliest completion date.\nB. It has the largest amount of float.\nC. It has the most activities in the PND.\nD. It is the path with the most expensive project activities.\n☑ A. Of all the choices presented, A is the best description of the critical path. The critical path\nis the path with the longest duration and it tells the earliest and the latest day the project can be\ncompleted. There can be instances, however, when the project’s expected end date is well beyond\nthe duration of the scheduled work. In such cases, the critical path is considered the path with the\nleast amount of float.\n☐✗\nB, C, and D are incorrect. These are false descriptions of the critical path. The critical path\nhas the smallest amount of float, is the longest duration, and does not necessarily have the most\nexpensive activities.\n21. You are the project manager for a construction project. Your foreman informs you that, because\nof the humidity, the concrete will need to cure for an additional 24 hours before the framing can\nbegin. To accommodate the requirement, you add _______________ time to the framing activity.\nA. Lead\nB. Lag\nC. Delay\nD. Slack\n☑ B. You will add lag time to the framing activity. Lag is waiting time.\n☐✗\nA, C, and D are incorrect. A is incorrect; lead time allows activities to overlap. C is not the\ncorrect choice, because a delay isn’t the correct nomenclature for the scenario. Lag time is what’s\nadded to the schedule, not a delay. D is also incorrect because slack is the amount of time a task\ncan be delayed without delaying the scheduled start date of dependent activities.\n\n\n\n22. Management has informed you that you must flatten your project through resource leveling.\nWhat is likely to happen to your project schedule if your project team members are allowed to\ncontribute only 30 hours per week?\nA. The project schedule will increase.\nB. The project schedule will decrease.\nC. The project critical path will change.\nD. The project manager will need to use the critical chain method.\n☑ A. When the project schedule is flattened through resource leveling, the project duration\n(schedule) will likely increase.\n☐✗\nB, C, and D are incorrect. The project schedule will not decrease, and there’s no evidence\nthat the critical path will change. The project manager can use the critical chain method or not,\nand it will likely not affect the project duration.\n23. You are the project manager for a project with the following network diagram. Study the diagram:\nWhich path is the critical path?\n4 5\n6\nA B 2\nC\nD\nStart\n5 4 1 End\nE F H\n4\nG\nA. ABCD\nB. EBCD\nC. EFH\nD. EGH\n☑ B is the critical path because EBCD is the longest path to completion at 18 days.\n☐✗\nA, C, and D are incorrect. These paths have float.\n\n\n\n24. Bertha is the project manager for the HAR Project. The project is behind schedule, and Bertha\nhas elected, with management’s approval, to crash the critical path. This process adds more what?\n(Choose the best answer.)\nA. Cost\nB. Time\nC. Risk\nD. Documentation\n☑ A. Crashing involves adding resources, which typically increases cost.\n☐✗\nB, C, and D are incorrect. B is incorrect because crashing is an effort to reduce time, not\nadd it. C may be correct, but it is not the best answer. D is incorrect because crashing doesn’t\nnecessarily add more documentation. Crashing adds resources, which will add costs to the project.\n25. Bertha is the project manager for the HAR Project. It’s currently behind schedule, and Bertha has\nelected, with management’s approval, to fast track the critical path. This process adds more what?\n(Choose the best answer.)\nA. Cost\nB. Time\nC. Risk\nD. Documentation\n☑ C. Fast tracking adds risk because tasks can overlap.\n☐✗\nA, B, and D are incorrect. A may be correct in some instances, but it is not the best choice\nhere. B is incorrect because Bertha wants to remove time, not add it. D is also incorrect, because\nfast tracking doesn’t necessarily add documentation to the project. Fast tracking allows phases\nand activities to overlap and this increases risk for the project.\n'
#print(all_text)

In [4]:
section = r"^\d+\.\s+([\S\s]*?)(?=\Z|^\d+\.\s+)"
# capture sections from question to question
sections = re.finditer(section, all_text, re.MULTILINE)

qnumber = r"(^\d+)\.\s+[\S\s]"
qnumbers = re.finditer(qnumber, all_text, re.MULTILINE)

question = r"\d+\.\s+([\S\s]*?)\nA\."
questions = re.finditer(question, all_text, re.MULTILINE)

choiceA = r"\nA\.([\S\s]*?)\nB\."
choiceAs = re.finditer(choiceA, all_text, re.MULTILINE)

choiceB = r"\nB\.([\S\s]*?)\nC\."
choiceBs = re.finditer(choiceB, all_text, re.MULTILINE)

choiceC = r"\nC\.([\S\s]*?)\nD\."
choiceCs = re.finditer(choiceC, all_text, re.MULTILINE)

choiceD = r"\nD\.([\S\s]*?)\n"
choiceDs = re.finditer(choiceD, all_text, re.MULTILINE)

answer_dict={}
    
answer_key_regex = r"☑\s*([ABCD])\.\s*(.*?)\n"

matches = re.findall(answer_key_regex, all_text, re.DOTALL)

i=1
for match in matches:
    answer = match[0]
    explanation = match[1].strip()
    answer_dict[str(i)]=answer
    i+=1
answer_dict

{'1': 'A',
 '2': 'B',
 '3': 'B',
 '4': 'A',
 '5': 'B',
 '6': 'A',
 '7': 'C',
 '8': 'D',
 '9': 'B',
 '10': 'D',
 '11': 'C',
 '12': 'A',
 '13': 'B',
 '14': 'A',
 '15': 'C',
 '16': 'A',
 '17': 'A',
 '18': 'B',
 '19': 'C',
 '20': 'A',
 '21': 'B',
 '22': 'A',
 '23': 'A',
 '24': 'C'}

In [5]:
explanation_dict={}
explanation_regex = r"☑\s*[ABCD]\.\s*(.*?)\n(?:☐✗\n(.*?\n)|$)"

matches = re.findall(explanation_regex, all_text, re.DOTALL)
i=1
for match in matches:
    explanation_correct = match[0].strip()
    explanation_incorrect = match[1].strip() if match[1] else ""
    explanation_dict[str(i)]=f"{explanation_correct} {explanation_incorrect}"
    i+=1
explanation_dict

{'1': 'The resource calendar is the best choice for this scenario because it is the only activity\nduration estimating input listed. B, C, and D are incorrect. B is incorrect because the order of magnitude provides little',
 '2': 'Before the work can begin, the certification class external to the work must be completed. A, C, and D are incorrect. A is incorrect; hard logic is the mandatory sequencing of events.',
 '3': 'The time between the activities is lag time. The painting activity must wait 36 hours\nbefore it can begin. A, C, and D are incorrect. A, lead time, is when the activities are brought closer together',
 '4': 'G is slated to start immediately after F, so this is a finish-to-start (FS) relationship. In other\nwords, F must finish so G may start. B, C, and D are incorrect. These relationships do not fit the scenario.',
 '5': 'The critical path is always the path with the longest duration; it’s also the minimum\namount of time needed to complete the project. A, C, and D are

In [14]:
import re
import unicodedata  # Module Unicode

matchesS = re.finditer(section, all_text, re.MULTILINE)
pmpqa = []

for sectionNum, sectionMatch in enumerate(matchesS):
    matchesQN = re.findall(qnumber, sectionMatch.group(0), re.MULTILINE)
    matchesQ = re.findall(question, sectionMatch.group(0), re.MULTILINE)
    matchesA = re.findall(choiceA, sectionMatch.group(0), re.MULTILINE)
    matchesB = re.findall(choiceB, sectionMatch.group(0), re.MULTILINE)
    matchesC = re.findall(choiceC, sectionMatch.group(0), re.MULTILINE)
    matchesD = re.findall(choiceD, sectionMatch.group(0), re.MULTILINE)
    
    for matchQN, matchQ, matchA, matchB, matchC, matchD in zip(matchesQN, matchesQ, matchesA, matchesB, matchesC, matchesD):
        # Normalize Unicode characters to their closest ASCII representation
        # This removes accents and converts special characters like '\u2018' and '\u2019' to standard ASCII equivalents.
        matchQ = unicodedata.normalize('NFKD', matchQ).encode('ascii', 'ignore').decode('utf-8')
        matchA = unicodedata.normalize('NFKD', matchA).encode('ascii', 'ignore').decode('utf-8')
        matchB = unicodedata.normalize('NFKD', matchB).encode('ascii', 'ignore').decode('utf-8')
        matchC = unicodedata.normalize('NFKD', matchC).encode('ascii', 'ignore').decode('utf-8')
        matchD = unicodedata.normalize('NFKD', matchD).encode('ascii', 'ignore').decode('utf-8')
        
        qadata = {}
        qadata['source'] = 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education'
        qadata['year'] = 2018
        qadata['type'] = 'study guide'
        qadata['qid'] = matchQN
        qadata['question'] = {"stem": matchQ.replace('\n',' ').strip(),
                              "choices": [
                                  {"label": "A", "text": matchA.replace('\n',' ').strip()},
                                  {"label": "B", "text": matchB.replace('\n',' ').strip()},
                                  {"label": "C", "text": matchC.replace('\n',' ').strip()},
                                  {"label": "D", "text": matchD.replace('\n',' ').strip()}]}
    
        
        #if matchQN in answer_dict.keys():
        if matchQN in ['21','23','4']:
            break
        qadata['answerKey'] = answer_dict.get(matchQN)
        
        statements = []
        for choice, text in zip(['A', 'B', 'C', 'D'], [matchA, matchB, matchC, matchD]):
            is_correct = (answer_dict.get(matchQN) == choice)
            statements.append({
                "label": is_correct,
                "statement": matchQ.replace('\n',' ').strip() + ' ' + text.replace('\n', ' ').strip()
            })
        #qadata['explanation'] = explanation_dict.get(matchQN)
        
        explanation = explanation_dict.get(matchQN)
        if explanation is not None:
            exp = unicodedata.normalize('NFKD', explanation).encode('ascii', 'ignore').decode('utf-8')
            qadata['explanation'] = exp
        else:
            qadata['explanation'] = None
        
        qadata['statements'] = statements
        pmpqa.append(qadata)

pmpqa


[{'source': 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education',
  'year': 2018,
  'type': 'study guide',
  'qid': '1',
  'question': {'stem': 'You are the project manager of the JHG Project. This project has 32 stakeholders and will require implementation activities in North and South America. You have been requested to provide a duration estimate for the project. Of the following, which will offer the best level of detail in your estimate?',
   'choices': [{'label': 'A', 'text': 'The resource calendar'},
    {'label': 'B', 'text': 'An order of magnitude'},
    {'label': 'C', 'text': 'A requirements document'},
    {'label': 'D', 'text': 'A stakeholder analysis'}]},
  'answerKey': 'A',
  'explanation': 'The resource calendar is the best choice for this scenario because it is the only activity\nduration estimating input listed. B, C, and D are incorrect. B is incorrect because the order of magnitude provides little',
  'statements': [{'label': True

In [15]:
# save the PMP-QA dataset as jsonl
with open(output_file, 'w') as fout:
    for dic in pmpqa:
        fout.write(json.dumps(dic) + '\n')

In [1]:
# import libraries
import re
import pdfplumber
import json

In [2]:
# define PDF file path
file_name = 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education (2018)'
input_file = file_name + '.pdf'
output_file = file_name + '_chapter7_statements.jsonl'

In [3]:
pages_to_extract = list(range(434, 443))

    
all_text = ""

with pdfplumber.open(input_file) as pdf:
    for page_num, page in enumerate(pdf.pages):
        if page_num + 1 in pages_to_extract:
            page_text = page.extract_text()
            if page_text:
                all_text += '\n' + page_text
pattern1 = r"CertPrs_2015/PMP® Project Management Professional Study Guide/Joseph Phillips/86198-8/Chapter \d+"
all_text=re.sub(pattern1, '', all_text)
pattern2 = r"Self Test Answers \d+"
all_text=re.sub(pattern2, '', all_text)
pattern3 = r"SELF TEST ANSWERS"
all_text=re.sub(pattern3, '', all_text)
pattern4 = r"\d+ Chapter 7 Introducing Project Cost Management"
all_text=re.sub(pattern4, '', all_text)
pattern5 = r"07-ch07.indd \d+ 02/11/17 1:53 PM"
all_text=re.sub(pattern5, '', all_text)   
#print(all_text)

In [4]:
section = r"^\d+\.\s+([\S\s]*?)(?=\Z|^\d+\.\s+)"
# capture sections from question to question
sections = re.finditer(section, all_text, re.MULTILINE)

qnumber = r"(^\d+)\.\s+[\S\s]"
qnumbers = re.finditer(qnumber, all_text, re.MULTILINE)

question = r"\d+\.\s+([\S\s]*?)\nA\."
questions = re.finditer(question, all_text, re.MULTILINE)

choiceA = r"\nA\.([\S\s]*?)\nB\."
choiceAs = re.finditer(choiceA, all_text, re.MULTILINE)

choiceB = r"\nB\.([\S\s]*?)\nC\."
choiceBs = re.finditer(choiceB, all_text, re.MULTILINE)

choiceC = r"\nC\.([\S\s]*?)\nD\."
choiceCs = re.finditer(choiceC, all_text, re.MULTILINE)

choiceD = r"\nD\.([\S\s]*?)\n"
choiceDs = re.finditer(choiceD, all_text, re.MULTILINE)

answer_dict={}
    
answer_key_regex = r"☑\s*([ABCD])\.\s*(.*?)\n"

matches = re.findall(answer_key_regex, all_text, re.DOTALL)

i=1
for match in matches:
    answer = match[0]
    explanation = match[1].strip()
    answer_dict[str(i)]=answer
    i+=1
answer_dict

{'1': 'C',
 '2': 'B',
 '3': 'A',
 '4': 'C',
 '5': 'D',
 '6': 'C',
 '7': 'A',
 '8': 'A',
 '9': 'B',
 '10': 'B',
 '11': 'D',
 '12': 'B',
 '13': 'A',
 '14': 'C',
 '15': 'C',
 '16': 'B',
 '17': 'C',
 '18': 'A',
 '19': 'C',
 '20': 'B',
 '21': 'C',
 '22': 'A',
 '23': 'B',
 '24': 'A'}

In [5]:
explanation_dict={}
explanation_regex = r"☑\s*[ABCD]\.\s*(.*?)\n(?:☐✗\n(.*?\n)|$)"

matches = re.findall(explanation_regex, all_text, re.DOTALL)
i=1
for match in matches:
    explanation_correct = match[0].strip()
    explanation_incorrect = match[1].strip() if match[1] else ""
    explanation_dict[str(i)]=f"{explanation_correct} {explanation_incorrect}"
    i+=1
explanation_dict

{'1': 'Analogous estimating is less accurate than other estimating methods. A, B, and D are incorrect. A is incorrect because regression analysis is a type of parametric',
 '2': 'The WBS is the input that can help you the most with the cost estimates. A, C, and D are incorrect. A is incorrect because parametric modeling is a form of estimating,',
 '3': 'The answer, $750 per ton, is an example of parametric estimating. B, C, and D are incorrect. B is incorrect because historical information is analogous, not',
 '4': 'Bottom-up estimating provides the most accurate estimates. The project manager starts\nat zero, the bottom, and accounts for each cost within the project. A, B, and D are incorrect. They do not reflect the most accurate methods to create an estimate.',
 '5': 'As the project team completes more and more units, the time to complete a hotel room\nshould take less and less time. D is the best choice. A, B, and C are incorrect because these choices do not reflect that learning c

In [7]:
import re
import unicodedata  # Module Unicode

matchesS = re.finditer(section, all_text, re.MULTILINE)
pmpqa = []

for sectionNum, sectionMatch in enumerate(matchesS):
    matchesQN = re.findall(qnumber, sectionMatch.group(0), re.MULTILINE)
    matchesQ = re.findall(question, sectionMatch.group(0), re.MULTILINE)
    matchesA = re.findall(choiceA, sectionMatch.group(0), re.MULTILINE)
    matchesB = re.findall(choiceB, sectionMatch.group(0), re.MULTILINE)
    matchesC = re.findall(choiceC, sectionMatch.group(0), re.MULTILINE)
    matchesD = re.findall(choiceD, sectionMatch.group(0), re.MULTILINE)
    
    for matchQN, matchQ, matchA, matchB, matchC, matchD in zip(matchesQN, matchesQ, matchesA, matchesB, matchesC, matchesD):
        # Normalize Unicode characters to their closest ASCII representation
        # This removes accents and converts special characters like '\u2018' and '\u2019' to standard ASCII equivalents.
        matchQ = unicodedata.normalize('NFKD', matchQ).encode('ascii', 'ignore').decode('utf-8')
        matchA = unicodedata.normalize('NFKD', matchA).encode('ascii', 'ignore').decode('utf-8')
        matchB = unicodedata.normalize('NFKD', matchB).encode('ascii', 'ignore').decode('utf-8')
        matchC = unicodedata.normalize('NFKD', matchC).encode('ascii', 'ignore').decode('utf-8')
        matchD = unicodedata.normalize('NFKD', matchD).encode('ascii', 'ignore').decode('utf-8')
        
        qadata = {}
        qadata['source'] = 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education'
        qadata['year'] = 2018
        qadata['type'] = 'study guide'
        qadata['qid'] = matchQN
        qadata['question'] = {"stem": matchQ.replace('\n',' ').strip(),
                              "choices": [
                                  {"label": "A", "text": matchA.replace('\n',' ').strip()},
                                  {"label": "B", "text": matchB.replace('\n',' ').strip()},
                                  {"label": "C", "text": matchC.replace('\n',' ').strip()},
                                  {"label": "D", "text": matchD.replace('\n',' ').strip()}]}
    
        
        #if matchQN in answer_dict.keys():
        qadata['answerKey'] = answer_dict.get(matchQN)
        
        statements = []
        for choice, text in zip(['A', 'B', 'C', 'D'], [matchA, matchB, matchC, matchD]):
            is_correct = (answer_dict.get(matchQN) == choice)
            statements.append({
                "label": is_correct,
                "statement": matchQ.replace('\n',' ').strip() + ' ' + text.replace('\n', ' ').strip()
            })
        #qadata['explanation'] = explanation_dict.get(matchQN)
        explanation = explanation_dict.get(matchQN)
        if explanation is not None:
            exp = unicodedata.normalize('NFKD', explanation).encode('ascii', 'ignore').decode('utf-8')
            qadata['explanation'] = exp
        else:
            qadata['explanation'] = None
        qadata['statements'] = statements
        pmpqa.append(qadata)

pmpqa


[{'source': 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education',
  'year': 2018,
  'type': 'study guide',
  'qid': '1',
  'question': {'stem': 'You are the project manager for your organization and you need to create a cost estimate for your current project. This project is similar to the ABG Project you completed a few months ago, so you report to your program manager that youll be using analogous estimating. Which of the following best describes analogous estimating?',
   'choices': [{'label': 'A', 'text': 'Regression analysis'},
    {'label': 'B', 'text': 'Bottom-up estimating'},
    {'label': 'C', 'text': 'Less accurate'},
    {'label': 'D', 'text': 'More accurate'}]},
  'answerKey': 'C',
  'explanation': 'Analogous estimating is less accurate than other estimating methods. A, B, and D are incorrect. A is incorrect because regression analysis is a type of parametric',
  'statements': [{'label': False,
    'statement': 'You are the project manag

In [8]:
# save the PMP-QA dataset as jsonl
with open(output_file, 'w') as fout:
    for dic in pmpqa:
        fout.write(json.dumps(dic) + '\n')

In [1]:
# import libraries
import re
import pdfplumber
import json

In [2]:
# define PDF file path
file_name = 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education (2018)'
input_file = file_name + '.pdf'
output_file = file_name + '_chapter8_statements.jsonl'

In [3]:
pages_to_extract = list(range(482, 492))

    
all_text = ""

with pdfplumber.open(input_file) as pdf:
    for page_num, page in enumerate(pdf.pages):
        if page_num + 1 in pages_to_extract:
            page_text = page.extract_text()
            if page_text:
                all_text += '\n' + page_text
pattern1 = r"CertPrs_2015/PMP® Project Management Professional Study Guide/Joseph Phillips/86198-8/Chapter \d+"
all_text=re.sub(pattern1, '', all_text)
pattern2 = r"Self Test Answers \d+"
all_text=re.sub(pattern2, '', all_text)
pattern3 = r"SELF TEST ANSWERS"
all_text=re.sub(pattern3, '', all_text)
pattern4 = r"\d+ Chapter 8 Introducing Project Quality Management"
all_text=re.sub(pattern4, '', all_text)
pattern5 = r"08-ch08.indd \d+ 02/11/17 1:59 PM"
all_text=re.sub(pattern5, '', all_text)   
#print(all_text)

In [4]:
section = r"^\d+\.\s+([\S\s]*?)(?=\Z|^\d+\.\s+)"
# capture sections from question to question
sections = re.finditer(section, all_text, re.MULTILINE)

qnumber = r"(^\d+)\.\s+[\S\s]"
qnumbers = re.finditer(qnumber, all_text, re.MULTILINE)

question = r"\d+\.\s+([\S\s]*?)\nA\."
questions = re.finditer(question, all_text, re.MULTILINE)

choiceA = r"\nA\.([\S\s]*?)\nB\."
choiceAs = re.finditer(choiceA, all_text, re.MULTILINE)

choiceB = r"\nB\.([\S\s]*?)\nC\."
choiceBs = re.finditer(choiceB, all_text, re.MULTILINE)

choiceC = r"\nC\.([\S\s]*?)\nD\."
choiceCs = re.finditer(choiceC, all_text, re.MULTILINE)

choiceD = r"\nD\.([\S\s]*?)\n"
choiceDs = re.finditer(choiceD, all_text, re.MULTILINE)

answer_dict={}
    
answer_key_regex = r"☑\s*([ABCD])\.\s*(.*?)\n"

matches = re.findall(answer_key_regex, all_text, re.DOTALL)

i=1
for match in matches:
    answer = match[0]
    explanation = match[1].strip()
    answer_dict[str(i)]=answer
    i+=1
answer_dict

{'1': 'B',
 '2': 'A',
 '3': 'A',
 '4': 'C',
 '5': 'A',
 '6': 'A',
 '7': 'A',
 '8': 'D',
 '9': 'A',
 '10': 'C',
 '11': 'A',
 '12': 'C',
 '13': 'A',
 '14': 'C',
 '15': 'D',
 '16': 'A',
 '17': 'C',
 '18': 'B',
 '19': 'C',
 '20': 'D',
 '21': 'C',
 '22': 'C',
 '23': 'A',
 '24': 'C',
 '25': 'C'}

In [5]:
explanation_dict={}
explanation_regex = r"☑\s*[ABCD]\.\s*(.*?)\n(?:☐✗\n(.*?\n)|$)"

matches = re.findall(explanation_regex, all_text, re.DOTALL)
i=1
for match in matches:
    explanation_correct = match[0].strip()
    explanation_incorrect = match[1].strip() if match[1] else ""
    explanation_dict[str(i)]=f"{explanation_correct} {explanation_incorrect}"
    i+=1
explanation_dict

{'1': 'The project team (the individuals completing the project work) is responsible for the\nquality of the project deliverables. A, C, and D are incorrect. A is incorrect because the project champion may review the',
 '2': 'The chart shown is a control chart. B, C, and D are incorrect. B is incorrect because a Pareto diagram maps categories of issues',
 '3': 'QA is concerned with overall project quality performance. B, C, and D are incorrect. They do not correctly explain quality assurance.',
 '4': 'Of all the choices presented, this is the best answer. Quality is planned into the project,\nand the planning requires time. A, B, and D are incorrect. A is incorrect because a project of this nature has been completed',
 '5': 'Quality planning should be completed prior to the work beginning—and should\nthereafter be revisited as needed. B, C, and D are incorrect. B is incorrect because quality management is not an applicable',
 '6': 'Quality planning is the process of creating a plan to 

In [8]:
import re
import unicodedata  # Module Unicode

matchesS = re.finditer(section, all_text, re.MULTILINE)
pmpqa = []

for sectionNum, sectionMatch in enumerate(matchesS):
    matchesQN = re.findall(qnumber, sectionMatch.group(0), re.MULTILINE)
    matchesQ = re.findall(question, sectionMatch.group(0), re.MULTILINE)
    matchesA = re.findall(choiceA, sectionMatch.group(0), re.MULTILINE)
    matchesB = re.findall(choiceB, sectionMatch.group(0), re.MULTILINE)
    matchesC = re.findall(choiceC, sectionMatch.group(0), re.MULTILINE)
    matchesD = re.findall(choiceD, sectionMatch.group(0), re.MULTILINE)
    
    for matchQN, matchQ, matchA, matchB, matchC, matchD in zip(matchesQN, matchesQ, matchesA, matchesB, matchesC, matchesD):
        # Normalize Unicode characters to their closest ASCII representation
        # This removes accents and converts special characters like '\u2018' and '\u2019' to standard ASCII equivalents.
        matchQ = unicodedata.normalize('NFKD', matchQ).encode('ascii', 'ignore').decode('utf-8')
        matchA = unicodedata.normalize('NFKD', matchA).encode('ascii', 'ignore').decode('utf-8')
        matchB = unicodedata.normalize('NFKD', matchB).encode('ascii', 'ignore').decode('utf-8')
        matchC = unicodedata.normalize('NFKD', matchC).encode('ascii', 'ignore').decode('utf-8')
        matchD = unicodedata.normalize('NFKD', matchD).encode('ascii', 'ignore').decode('utf-8')
        
        qadata = {}
        qadata['source'] = 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education'
        qadata['year'] = 2018
        qadata['type'] = 'study guide'
        qadata['qid'] = matchQN
        qadata['question'] = {"stem": matchQ.replace('\n',' ').strip(),
                              "choices": [
                                  {"label": "A", "text": matchA.replace('\n',' ').strip()},
                                  {"label": "B", "text": matchB.replace('\n',' ').strip()},
                                  {"label": "C", "text": matchC.replace('\n',' ').strip()},
                                  {"label": "D", "text": matchD.replace('\n',' ').strip()}]}
    
        
        #if matchQN in answer_dict.keys():
        if matchQN in ['8','9','13']:
            break
        qadata['answerKey'] = answer_dict.get(matchQN)
        
        statements = []
        for choice, text in zip(['A', 'B', 'C', 'D'], [matchA, matchB, matchC, matchD]):
            is_correct = (answer_dict.get(matchQN) == choice)
            statements.append({
                "label": is_correct,
                "statement": matchQ.replace('\n',' ').strip() + ' ' + text.replace('\n', ' ').strip()
            })
        explanation = explanation_dict.get(matchQN)
        if explanation is not None:
            exp = unicodedata.normalize('NFKD', explanation).encode('ascii', 'ignore').decode('utf-8')
            qadata['explanation'] = exp
        else:
            qadata['explanation'] = None
        qadata['statements'] = statements
        pmpqa.append(qadata)

pmpqa


[{'source': 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education',
  'year': 2018,
  'type': 'study guide',
  'qid': '1',
  'question': {'stem': 'Gary is the project manager of the HBB Project for his organization. Hes working with project customers to define the specifics of the project requirements, the project scope, and the defined product scope to help define the quality of the project. The customer asks Gary who is responsible for the quality of the project deliverables. Which of the following is responsible for managing quality?',
   'choices': [{'label': 'A', 'text': 'The project champion'},
    {'label': 'B', 'text': 'The project team'},
    {'label': 'C', 'text': 'Stakeholders'},
    {'label': 'D', 'text': 'Everyone'}]},
  'answerKey': 'B',
  'explanation': 'The project team (the individuals completing the project work) is responsible for the\nquality of the project deliverables. A, C, and D are incorrect. A is incorrect because the project

In [9]:
# save the PMP-QA dataset as jsonl
with open(output_file, 'w') as fout:
    for dic in pmpqa:
        fout.write(json.dumps(dic) + '\n')

In [1]:
# import libraries
import re
import pdfplumber
import json

In [2]:
# define PDF file path
file_name = 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education (2018)'
input_file = file_name + '.pdf'
output_file = file_name + '_chapter9_statements.jsonl'

In [3]:
pages_to_extract = list(range(538, 549))

    
all_text = ""

with pdfplumber.open(input_file) as pdf:
    for page_num, page in enumerate(pdf.pages):
        if page_num + 1 in pages_to_extract:
            page_text = page.extract_text()
            if page_text:
                all_text += '\n' + page_text
pattern1 = r"CertPrs_2015/PMP® Project Management Professional Study Guide/Joseph Phillips/86198-8/Chapter \d+"
all_text=re.sub(pattern1, '', all_text)
pattern2 = r"Self Test Answers \d+"
all_text=re.sub(pattern2, '', all_text)
pattern3 = r"SELF TEST ANSWERS"
all_text=re.sub(pattern3, '', all_text)
pattern4 = r"\d+ Chapter 9 Introducing Project Resource Management"
all_text=re.sub(pattern4, '', all_text)
pattern5 = r"09-ch09.indd \d+ 02/11/17 2:09 PM"
all_text=re.sub(pattern5, '', all_text)   
#print(all_text)

In [4]:
section = r"^\d+\.\s+([\S\s]*?)(?=\Z|^\d+\.\s+)"
# capture sections from question to question
sections = re.finditer(section, all_text, re.MULTILINE)

qnumber = r"(^\d+)\.\s+[\S\s]"
qnumbers = re.finditer(qnumber, all_text, re.MULTILINE)

question = r"\d+\.\s+([\S\s]*?)\nA\."
questions = re.finditer(question, all_text, re.MULTILINE)

choiceA = r"\nA\.([\S\s]*?)\nB\."
choiceAs = re.finditer(choiceA, all_text, re.MULTILINE)

choiceB = r"\nB\.([\S\s]*?)\nC\."
choiceBs = re.finditer(choiceB, all_text, re.MULTILINE)

choiceC = r"\nC\.([\S\s]*?)\nD\."
choiceCs = re.finditer(choiceC, all_text, re.MULTILINE)

choiceD = r"\nD\.([\S\s]*?)\n"
choiceDs = re.finditer(choiceD, all_text, re.MULTILINE)

answer_dict={}
    
answer_key_regex = r"☑\s*([ABCD])\.\s*(.*?)\n"

matches = re.findall(answer_key_regex, all_text, re.DOTALL)

i=1
for match in matches:
    answer = match[0]
    explanation = match[1].strip()
    answer_dict[str(i)]=answer
    i+=1
answer_dict

{'1': 'A',
 '2': 'B',
 '3': 'B',
 '4': 'C',
 '5': 'C',
 '6': 'B',
 '7': 'C',
 '8': 'C',
 '9': 'B',
 '10': 'B',
 '11': 'A',
 '12': 'B',
 '13': 'A',
 '14': 'A',
 '15': 'D',
 '16': 'D',
 '17': 'B',
 '18': 'D',
 '19': 'C',
 '20': 'D',
 '21': 'B',
 '22': 'C',
 '23': 'C',
 '24': 'A',
 '25': 'C'}

In [5]:
explanation_dict={}
explanation_regex = r"☑\s*[ABCD]\.\s*(.*?)\n(?:☐✗\n(.*?\n)|$)"

matches = re.findall(explanation_regex, all_text, re.DOTALL)
i=1
for match in matches:
    explanation_correct = match[0].strip()
    explanation_incorrect = match[1].strip() if match[1] else ""
    explanation_dict[str(i)]=f"{explanation_correct} {explanation_incorrect}"
    i+=1
explanation_dict

{'1': 'With just-in-time manufacturing, the physical resources are put in place only as they\nare needed. This approach reduces waste, keeps inventory at a minimum, and helps the project\nmanager forecast resource utilization more accurately. B, C, and D are incorrect. B, schedule ALAP, means to schedule as late as possible to',
 '2': 'Because the project requires the electrician, a project role, this is a resource requirement. A, C, and D are incorrect. A is incorrect because it does not accurately describe the',
 '3': 'The project needs the resource of the chemical engineer to be successful. When the\nproject needs a resource, it is a resource requirement. A, C, and D are incorrect. This situation does not describe an organizational interface or',
 '4': 'In a weak matrix structure, functional management will have more authority than the\nproject manager. A, B, and D are incorrect. They do not have as much authority on a project in a weak matrix',
 '5': 'In this instance, the union is

In [6]:
import re
import unicodedata  # Module Unicode

matchesS = re.finditer(section, all_text, re.MULTILINE)
pmpqa = []

for sectionNum, sectionMatch in enumerate(matchesS):
    matchesQN = re.findall(qnumber, sectionMatch.group(0), re.MULTILINE)
    matchesQ = re.findall(question, sectionMatch.group(0), re.MULTILINE)
    matchesA = re.findall(choiceA, sectionMatch.group(0), re.MULTILINE)
    matchesB = re.findall(choiceB, sectionMatch.group(0), re.MULTILINE)
    matchesC = re.findall(choiceC, sectionMatch.group(0), re.MULTILINE)
    matchesD = re.findall(choiceD, sectionMatch.group(0), re.MULTILINE)
    
    for matchQN, matchQ, matchA, matchB, matchC, matchD in zip(matchesQN, matchesQ, matchesA, matchesB, matchesC, matchesD):
        # Normalize Unicode characters to their closest ASCII representation
        # This removes accents and converts special characters like '\u2018' and '\u2019' to standard ASCII equivalents.
        matchQ = unicodedata.normalize('NFKD', matchQ).encode('ascii', 'ignore').decode('utf-8')
        matchA = unicodedata.normalize('NFKD', matchA).encode('ascii', 'ignore').decode('utf-8')
        matchB = unicodedata.normalize('NFKD', matchB).encode('ascii', 'ignore').decode('utf-8')
        matchC = unicodedata.normalize('NFKD', matchC).encode('ascii', 'ignore').decode('utf-8')
        matchD = unicodedata.normalize('NFKD', matchD).encode('ascii', 'ignore').decode('utf-8')
        
        qadata = {}
        qadata['source'] = 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education'
        qadata['year'] = 2018
        qadata['type'] = 'study guide'
        qadata['qid'] = matchQN
        qadata['question'] = {"stem": matchQ.replace('\n',' ').strip(),
                              "choices": [
                                  {"label": "A", "text": matchA.replace('\n',' ').strip()},
                                  {"label": "B", "text": matchB.replace('\n',' ').strip()},
                                  {"label": "C", "text": matchC.replace('\n',' ').strip()},
                                  {"label": "D", "text": matchD.replace('\n',' ').strip()}]}
    
        
        #if matchQN in answer_dict.keys():
        qadata['answerKey'] = answer_dict.get(matchQN)
        
        statements = []
        for choice, text in zip(['A', 'B', 'C', 'D'], [matchA, matchB, matchC, matchD]):
            is_correct = (answer_dict.get(matchQN) == choice)
            statements.append({
                "label": is_correct,
                "statement": matchQ.replace('\n',' ').strip() + ' ' + text.replace('\n', ' ').strip()
            })
        explanation = explanation_dict.get(matchQN)
        if explanation is not None:
            exp = unicodedata.normalize('NFKD', explanation).encode('ascii', 'ignore').decode('utf-8')
            qadata['explanation'] = exp
        else:
            qadata['explanation'] = None
        qadata['statements'] = statements
        pmpqa.append(qadata)

pmpqa


[{'source': 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education',
  'year': 2018,
  'type': 'study guide',
  'qid': '1',
  'question': {'stem': 'You are the project manager for the JHG Project. This project requires coordination with the directors of manufacturing, resources, the IT department, and the CIO. You will be acquiring resources for the project only as the resources are needed. The goal is to reduce waste, minimize inventory, and forecast resource utilization. What type of approach is this?',
   'choices': [{'label': 'A', 'text': 'Just-in-time manufacturing'},
    {'label': 'B', 'text': 'Schedule ALAP'},
    {'label': 'C', 'text': 'Schedule ASAP'},
    {'label': 'D', 'text': 'Resource recurring factoring'}]},
  'answerKey': 'A',
  'explanation': 'With just-in-time manufacturing, the physical resources are put in place only as they\nare needed. This approach reduces waste, keeps inventory at a minimum, and helps the project\nmanager forecas

In [7]:
# save the PMP-QA dataset as jsonl
with open(output_file, 'w') as fout:
    for dic in pmpqa:
        fout.write(json.dumps(dic) + '\n')

In [1]:
# import libraries
import re
import pdfplumber
import json

In [2]:
# define PDF file path
file_name = 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education (2018)'
input_file = file_name + '.pdf'
output_file = file_name + '_10_statements.jsonl'

In [3]:
pages_to_extract = list(range(579, 589))

    
all_text = ""

with pdfplumber.open(input_file) as pdf:
    for page_num, page in enumerate(pdf.pages):
        if page_num + 1 in pages_to_extract:
            page_text = page.extract_text()
            if page_text:
                all_text += '\n' + page_text
pattern1 = r"CertPrs_2015/PMP® Project Management Professional Study Guide/Joseph Phillips/86198-8/Chapter \d+"
all_text=re.sub(pattern1, '', all_text)
pattern2 = r"Self Test Answers \d+"
all_text=re.sub(pattern2, '', all_text)
pattern3 = r"SELF TEST ANSWERS"
all_text=re.sub(pattern3, '', all_text)
pattern4 = r"\d+ Chapter 10 Introducing Project Communications Management"
all_text=re.sub(pattern4, '', all_text)
pattern5 =r"10-ch10.indd \d+ 02/11/17 2:15 PM"
all_text=re.sub(pattern5, '', all_text)   
#print(all_text)

In [4]:
section = r"^\d+\.\s+([\S\s]*?)(?=\Z|^\d+\.\s+)"
# capture sections from question to question
sections = re.finditer(section, all_text, re.MULTILINE)

qnumber = r"(^\d+)\.\s+[\S\s]"
qnumbers = re.finditer(qnumber, all_text, re.MULTILINE)

question = r"\d+\.\s+([\S\s]*?)\nA\."
questions = re.finditer(question, all_text, re.MULTILINE)

choiceA = r"\nA\.([\S\s]*?)\nB\."
choiceAs = re.finditer(choiceA, all_text, re.MULTILINE)

choiceB = r"\nB\.([\S\s]*?)\nC\."
choiceBs = re.finditer(choiceB, all_text, re.MULTILINE)

choiceC = r"\nC\.([\S\s]*?)\nD\."
choiceCs = re.finditer(choiceC, all_text, re.MULTILINE)

choiceD = r"\nD\.([\S\s]*?)\n"
choiceDs = re.finditer(choiceD, all_text, re.MULTILINE)

answer_dict={}
    
answer_key_regex = r"☑\s*([ABCD])\.\s*(.*?)\n"

matches = re.findall(answer_key_regex, all_text, re.DOTALL)

i=1
for match in matches:
    answer = match[0]
    explanation = match[1].strip()
    answer_dict[str(i)]=answer
    i+=1
answer_dict

{'1': 'D',
 '2': 'A',
 '3': 'C',
 '4': 'D',
 '5': 'B',
 '6': 'C',
 '7': 'A',
 '8': 'D',
 '9': 'D',
 '10': 'A',
 '11': 'C',
 '12': 'A',
 '13': 'B',
 '14': 'D',
 '15': 'C',
 '16': 'A',
 '17': 'C',
 '18': 'D',
 '19': 'D',
 '20': 'C',
 '21': 'C',
 '22': 'D',
 '23': 'D',
 '24': 'D',
 '25': 'A'}

In [5]:
explanation_dict={}
explanation_regex = r"☑\s*[ABCD]\.\s*(.*?)\n(?:☐✗\n(.*?\n)|$)"

matches = re.findall(explanation_regex, all_text, re.DOTALL)
i=1
for match in matches:
    explanation_correct = match[0].strip()
    explanation_incorrect = match[1].strip() if match[1] else ""
    explanation_dict[str(i)]=f"{explanation_correct} {explanation_incorrect}"
    i+=1
explanation_dict

{'1': 'Noise is anything that interferes with the transmission and understanding of the message.\nDistance is an example of noise. A, B, and C are incorrect. A, e-mail servers, is incorrect because it is an example of',
 '2': 'Acknowledgement means that Jose has received the message but may not agree with it. B, C, and D are incorrect. A transmission is the output of the message, negotiation is not',
 '3': 'Management is requesting a communication management plan, which details the\nrequirements and expectations for communicating information among the project stakeholders. A, B, and D are incorrect. A is incorrect because a roles and responsibilities matrix',
 '4': 'Lessons learned and historical information from a previous project are ideal inputs to\ncommunications planning. A, B, and C are incorrect. These answers do not fully answer the question. Lessons learned',
 '5': 'Using the formula N (N – 1) / 2, where N represents the number of stakeholders, you\nhave 496 communication chan

In [6]:
import re
import unicodedata  # Module Unicode

matchesS = re.finditer(section, all_text, re.MULTILINE)
pmpqa = []

for sectionNum, sectionMatch in enumerate(matchesS):
    matchesQN = re.findall(qnumber, sectionMatch.group(0), re.MULTILINE)
    matchesQ = re.findall(question, sectionMatch.group(0), re.MULTILINE)
    matchesA = re.findall(choiceA, sectionMatch.group(0), re.MULTILINE)
    matchesB = re.findall(choiceB, sectionMatch.group(0), re.MULTILINE)
    matchesC = re.findall(choiceC, sectionMatch.group(0), re.MULTILINE)
    matchesD = re.findall(choiceD, sectionMatch.group(0), re.MULTILINE)
    
    for matchQN, matchQ, matchA, matchB, matchC, matchD in zip(matchesQN, matchesQ, matchesA, matchesB, matchesC, matchesD):
        # Normalize Unicode characters to their closest ASCII representation
        # This removes accents and converts special characters like '\u2018' and '\u2019' to standard ASCII equivalents.
        matchQ = unicodedata.normalize('NFKD', matchQ).encode('ascii', 'ignore').decode('utf-8')
        matchA = unicodedata.normalize('NFKD', matchA).encode('ascii', 'ignore').decode('utf-8')
        matchB = unicodedata.normalize('NFKD', matchB).encode('ascii', 'ignore').decode('utf-8')
        matchC = unicodedata.normalize('NFKD', matchC).encode('ascii', 'ignore').decode('utf-8')
        matchD = unicodedata.normalize('NFKD', matchD).encode('ascii', 'ignore').decode('utf-8')
        
        qadata = {}
        qadata['source'] = 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education'
        qadata['year'] = 2018
        qadata['type'] = 'study guide'
        qadata['qid'] = matchQN
        qadata['question'] = {"stem": matchQ.replace('\n',' ').strip(),
                              "choices": [
                                  {"label": "A", "text": matchA.replace('\n',' ').strip()},
                                  {"label": "B", "text": matchB.replace('\n',' ').strip()},
                                  {"label": "C", "text": matchC.replace('\n',' ').strip()},
                                  {"label": "D", "text": matchD.replace('\n',' ').strip()}]}
    
        
        #if matchQN in answer_dict.keys():
        qadata['answerKey'] = answer_dict.get(matchQN)
        
        statements = []
        for choice, text in zip(['A', 'B', 'C', 'D'], [matchA, matchB, matchC, matchD]):
            is_correct = (answer_dict.get(matchQN) == choice)
            statements.append({
                "label": is_correct,
                "statement": matchQ.replace('\n',' ').strip() + ' ' + text.replace('\n', ' ').strip()
            })
        explanation = explanation_dict.get(matchQN)
        if explanation is not None:
            exp = unicodedata.normalize('NFKD', explanation).encode('ascii', 'ignore').decode('utf-8')
            qadata['explanation'] = exp
        else:
            qadata['explanation'] = None
        qadata['statements'] = statements
        pmpqa.append(qadata)

pmpqa


[{'source': 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education',
  'year': 2018,
  'type': 'study guide',
  'qid': '1',
  'question': {'stem': 'In the project communication model, many elements must be present to allow communication to occur. One element, noise, can prevent the effectiveness of the communication message. Of the following, which one is an example of noise?',
   'choices': [{'label': 'A', 'text': 'E-mail servers'},
    {'label': 'B', 'text': 'Ad hoc conversations'},
    {'label': 'C', 'text': 'Contractual agreements'},
    {'label': 'D', 'text': 'Distance'}]},
  'answerKey': 'D',
  'explanation': 'Noise is anything that interferes with the transmission and understanding of the message.\nDistance is an example of noise. A, B, and C are incorrect. A, e-mail servers, is incorrect because it is an example of',
  'statements': [{'label': False,
    'statement': 'In the project communication model, many elements must be present to allow co

In [8]:
# save the PMP-QA dataset as jsonl
with open(output_file, 'w') as fout:
    for dic in pmpqa:
        fout.write(json.dumps(dic) + '\n')

In [1]:
# import libraries
import re
import pdfplumber
import json

In [2]:
# define PDF file path
file_name = 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education (2018)'
input_file = file_name + '.pdf'
output_file = file_name + '_chapter11_statements.jsonl'

In [3]:
pages_to_extract = list(range(645, 656))

    
all_text = ""

with pdfplumber.open(input_file) as pdf:
    for page_num, page in enumerate(pdf.pages):
        if page_num + 1 in pages_to_extract:
            page_text = page.extract_text()
            if page_text:
                all_text += '\n' + page_text
pattern1 = r"CertPrs_2015/PMP® Project Management Professional Study Guide/Joseph Phillips/86198-8/Chapter \d+"
all_text=re.sub(pattern1, '', all_text)
pattern2 = r"Self Test Answers \d+"
all_text=re.sub(pattern2, '', all_text)
pattern3 = r"SELF TEST ANSWERS"
all_text=re.sub(pattern3, '', all_text)
pattern4 = r"\d+ Chapter 11 Introducing Project Risk Management"
all_text=re.sub(pattern4, '', all_text)
pattern5 =r"11-ch11.indd \d+ 02/11/17 2:22 PM"
all_text=re.sub(pattern5, '', all_text)   
#print(all_text)

In [4]:
section = r"^\d+\.\s+([\S\s]*?)(?=\Z|^\d+\.\s+)"
# capture sections from question to question
sections = re.finditer(section, all_text, re.MULTILINE)

qnumber = r"(^\d+)\.\s+[\S\s]"
qnumbers = re.finditer(qnumber, all_text, re.MULTILINE)

question = r"\d+\.\s+([\S\s]*?)\nA\."
questions = re.finditer(question, all_text, re.MULTILINE)

choiceA = r"\nA\.([\S\s]*?)\nB\."
choiceAs = re.finditer(choiceA, all_text, re.MULTILINE)

choiceB = r"\nB\.([\S\s]*?)\nC\."
choiceBs = re.finditer(choiceB, all_text, re.MULTILINE)

choiceC = r"\nC\.([\S\s]*?)\nD\."
choiceCs = re.finditer(choiceC, all_text, re.MULTILINE)

choiceD = r"\nD\.([\S\s]*?)\n"
choiceDs = re.finditer(choiceD, all_text, re.MULTILINE)

answer_dict={}
    
answer_key_regex = r"☑\s*([ABCD])\.\s*(.*?)\n"

matches = re.findall(answer_key_regex, all_text, re.DOTALL)

i=1
for match in matches:
    answer = match[0]
    explanation = match[1].strip()
    answer_dict[str(i)]=answer
    i+=1
answer_dict

{'1': 'D',
 '2': 'C',
 '3': 'C',
 '4': 'A',
 '5': 'A',
 '6': 'A',
 '7': 'B',
 '8': 'A',
 '9': 'B',
 '10': 'C',
 '11': 'D',
 '12': 'C',
 '13': 'C',
 '14': 'D',
 '15': 'B',
 '16': 'B',
 '17': 'B',
 '18': 'D',
 '19': 'C',
 '20': 'D',
 '21': 'B',
 '22': 'C',
 '23': 'A',
 '24': 'B',
 '25': 'B'}

In [5]:
explanation_dict={}
explanation_regex = r"☑\s*[ABCD]\.\s*(.*?)\n(?:☐✗\n(.*?\n)|$)"

matches = re.findall(explanation_regex, all_text, re.DOTALL)
i=1
for match in matches:
    explanation_correct = match[0].strip()
    explanation_incorrect = match[1].strip() if match[1] else ""
    explanation_dict[str(i)]=f"{explanation_correct} {explanation_incorrect}"
    i+=1
explanation_dict

{'1': 'Risks that are in balance with the reward are appropriate for acceptance. A, B, and C are incorrect. These solutions are all false responses to risk management. It',
 '2': 'Planning meetings are used to create the risk management plan. The project manager,\nproject team leaders, key stakeholders, and other individuals with the power to make decisions\nregarding risk management attend the meetings. A, B, and D are incorrect. These choices do not fully answer the question.',
 '3': 'The technical assessment board may be used as part of the change control system. It is\nnot relevant to risk management planning. A, B, and D are incorrect. A is incorrect because roles and responsibilities are a part of the',
 '4': 'An error value of 7 percent represents the threshold under which the project can operate.\nShould the number of errors increase beyond 7 percent, the current plastic will be substituted. B, C, and D are incorrect. B is incorrect because tracking is the documentation of a',


In [8]:
import re
import unicodedata  # Module Unicode

matchesS = re.finditer(section, all_text, re.MULTILINE)
pmpqa = []

for sectionNum, sectionMatch in enumerate(matchesS):
    matchesQN = re.findall(qnumber, sectionMatch.group(0), re.MULTILINE)
    matchesQ = re.findall(question, sectionMatch.group(0), re.MULTILINE)
    matchesA = re.findall(choiceA, sectionMatch.group(0), re.MULTILINE)
    matchesB = re.findall(choiceB, sectionMatch.group(0), re.MULTILINE)
    matchesC = re.findall(choiceC, sectionMatch.group(0), re.MULTILINE)
    matchesD = re.findall(choiceD, sectionMatch.group(0), re.MULTILINE)
    
    for matchQN, matchQ, matchA, matchB, matchC, matchD in zip(matchesQN, matchesQ, matchesA, matchesB, matchesC, matchesD):
        # Normalize Unicode characters to their closest ASCII representation
        # This removes accents and converts special characters like '\u2018' and '\u2019' to standard ASCII equivalents.
        matchQ = unicodedata.normalize('NFKD', matchQ).encode('ascii', 'ignore').decode('utf-8')
        matchA = unicodedata.normalize('NFKD', matchA).encode('ascii', 'ignore').decode('utf-8')
        matchB = unicodedata.normalize('NFKD', matchB).encode('ascii', 'ignore').decode('utf-8')
        matchC = unicodedata.normalize('NFKD', matchC).encode('ascii', 'ignore').decode('utf-8')
        matchD = unicodedata.normalize('NFKD', matchD).encode('ascii', 'ignore').decode('utf-8')
        
        qadata = {}
        qadata['source'] = 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education'
        qadata['year'] = 2018
        qadata['type'] = 'study guide'
        qadata['qid'] = matchQN
        qadata['question'] = {"stem": matchQ.replace('\n',' ').strip(),
                              "choices": [
                                  {"label": "A", "text": matchA.replace('\n',' ').strip()},
                                  {"label": "B", "text": matchB.replace('\n',' ').strip()},
                                  {"label": "C", "text": matchC.replace('\n',' ').strip()},
                                  {"label": "D", "text": matchD.replace('\n',' ').strip()}]}
    
        
        #if matchQN in answer_dict.keys():
        if matchQN in ['9','10','14','17']:
            break
        qadata['answerKey'] = answer_dict.get(matchQN)
        
        statements = []
        for choice, text in zip(['A', 'B', 'C', 'D'], [matchA, matchB, matchC, matchD]):
            is_correct = (answer_dict.get(matchQN) == choice)
            statements.append({
                "label": is_correct,
                "statement": matchQ.replace('\n',' ').strip() + ' ' + text.replace('\n', ' ').strip()
            })
        explanation = explanation_dict.get(matchQN)
        if explanation is not None:
            exp = unicodedata.normalize('NFKD', explanation).encode('ascii', 'ignore').decode('utf-8')
            qadata['explanation'] = exp
        else:
            qadata['explanation'] = None
        qadata['statements'] = statements
        pmpqa.append(qadata)

pmpqa


[{'source': 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education',
  'year': 2018,
  'type': 'study guide',
  'qid': '1',
  'question': {'stem': 'Beth is the project manager for her company, and Marty, her supervisor, is concerned with the possibility of Beth accepting one of the project risks. Beth explains that this risk should be accepted within the project. When is it appropriate to accept a project risk?',
   'choices': [{'label': 'A',
     'text': 'It is never appropriate to accept a project risk.'},
    {'label': 'B', 'text': 'All risks must be mitigated or transferred.'},
    {'label': 'C',
     'text': 'It is appropriate to accept a risk if the project team has never completed this type of project work before.'},
    {'label': 'D',
     'text': 'It is appropriate to accept a risk if the risk is in balance with the reward.'}]},
  'answerKey': 'D',
  'explanation': 'Risks that are in balance with the reward are appropriate for acceptance. A, B

In [9]:
# save the PMP-QA dataset as jsonl
with open(output_file, 'w') as fout:
    for dic in pmpqa:
        fout.write(json.dumps(dic) + '\n')

In [1]:
# import libraries
import re
import pdfplumber
import json

In [2]:
# define PDF file path
file_name = 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education (2018)'
input_file = file_name + '.pdf'
output_file = file_name + '_chapter12_statements.jsonl'

In [3]:
pages_to_extract = list(range(698, 708))

    
all_text = ""

with pdfplumber.open(input_file) as pdf:
    for page_num, page in enumerate(pdf.pages):
        if page_num + 1 in pages_to_extract:
            page_text = page.extract_text()
            if page_text:
                all_text += '\n' + page_text
pattern1 = r"CertPrs_2015/PMP® Project Management Professional Study Guide/Joseph Phillips/86198-8/Chapter \d+"
all_text=re.sub(pattern1, '', all_text)
pattern2 = r"Self Test Answers \d+"
all_text=re.sub(pattern2, '', all_text)
pattern3 = r"SELF TEST ANSWERS"
all_text=re.sub(pattern3, '', all_text)
pattern4 = r"\d+ Chapter 12 Introducing Project Procurement Management"
all_text=re.sub(pattern4, '', all_text)
pattern5 =r"12-ch12.indd \d+ 02/11/17 2:30 PM"
all_text=re.sub(pattern5, '', all_text)   
#print(all_text)

In [4]:
section = r"^\d+\.\s+([\S\s]*?)(?=\Z|^\d+\.\s+)"
# capture sections from question to question
sections = re.finditer(section, all_text, re.MULTILINE)

qnumber = r"(^\d+)\.\s+[\S\s]"
qnumbers = re.finditer(qnumber, all_text, re.MULTILINE)

question = r"\d+\.\s+([\S\s]*?)\nA\."
questions = re.finditer(question, all_text, re.MULTILINE)

choiceA = r"\nA\.([\S\s]*?)\nB\."
choiceAs = re.finditer(choiceA, all_text, re.MULTILINE)

choiceB = r"\nB\.([\S\s]*?)\nC\."
choiceBs = re.finditer(choiceB, all_text, re.MULTILINE)

choiceC = r"\nC\.([\S\s]*?)\nD\."
choiceCs = re.finditer(choiceC, all_text, re.MULTILINE)

choiceD = r"\nD\.([\S\s]*?)\n"
choiceDs = re.finditer(choiceD, all_text, re.MULTILINE)

answer_dict={}
    
answer_key_regex = r"☑\s*([ABCD])\.\s*(.*?)\n"

matches = re.findall(answer_key_regex, all_text, re.DOTALL)

i=1
for match in matches:
    answer = match[0]
    explanation = match[1].strip()
    answer_dict[str(i)]=answer
    i+=1
answer_dict

{'1': 'B',
 '2': 'B',
 '3': 'A',
 '4': 'C',
 '5': 'B',
 '6': 'C',
 '7': 'C',
 '8': 'B',
 '9': 'D',
 '10': 'C',
 '11': 'D',
 '12': 'C',
 '13': 'B',
 '14': 'B',
 '15': 'C',
 '16': 'B',
 '17': 'D',
 '18': 'B',
 '19': 'A',
 '20': 'B',
 '21': 'D',
 '22': 'B',
 '23': 'C',
 '24': 'B',
 '25': 'C'}

In [5]:
explanation_dict={}
explanation_regex = r"☑\s*[ABCD]\.\s*(.*?)\n(?:☐✗\n(.*?\n)|$)"

matches = re.findall(explanation_regex, all_text, re.DOTALL)
i=1
for match in matches:
    explanation_correct = match[0].strip()
    explanation_incorrect = match[1].strip() if match[1] else ""
    explanation_dict[str(i)]=f"{explanation_correct} {explanation_incorrect}"
    i+=1
explanation_dict

{'1': 'A contract can be used as a risk mitigation tool. Procurement of risky activities is known\nas transference—the risk does not disappear, but the responsibility for the risk is transferred to\nthe vendor. A, C, and D are incorrect. A vendor proposal, a quotation, and project requirements do',
 '2': 'A contract cannot include provisions for illegal activities. A, C, and D are incorrect. A is incorrect because a contract can stipulate a deadline for the',
 '3': 'Of all the answers presented, A is the best. Contracts have an offer and a consideration. B, C, and D are incorrect. B is incorrect because not all contracts demand signatures and',
 '4': 'The WBS defines the details and requirements for acceptance of the project. This\ninformation also serves as valuable input to the process of determining what needs to be\nprocured. The WBS defines the end result of the project. In dealing with vendors to procure a\nportion of the project, the work to be procured must support the requirem

In [6]:
import re
import unicodedata  # Module Unicode

matchesS = re.finditer(section, all_text, re.MULTILINE)
pmpqa = []

for sectionNum, sectionMatch in enumerate(matchesS):
    matchesQN = re.findall(qnumber, sectionMatch.group(0), re.MULTILINE)
    matchesQ = re.findall(question, sectionMatch.group(0), re.MULTILINE)
    matchesA = re.findall(choiceA, sectionMatch.group(0), re.MULTILINE)
    matchesB = re.findall(choiceB, sectionMatch.group(0), re.MULTILINE)
    matchesC = re.findall(choiceC, sectionMatch.group(0), re.MULTILINE)
    matchesD = re.findall(choiceD, sectionMatch.group(0), re.MULTILINE)
    
    for matchQN, matchQ, matchA, matchB, matchC, matchD in zip(matchesQN, matchesQ, matchesA, matchesB, matchesC, matchesD):
        # Normalize Unicode characters to their closest ASCII representation
        # This removes accents and converts special characters like '\u2018' and '\u2019' to standard ASCII equivalents.
        matchQ = unicodedata.normalize('NFKD', matchQ).encode('ascii', 'ignore').decode('utf-8')
        matchA = unicodedata.normalize('NFKD', matchA).encode('ascii', 'ignore').decode('utf-8')
        matchB = unicodedata.normalize('NFKD', matchB).encode('ascii', 'ignore').decode('utf-8')
        matchC = unicodedata.normalize('NFKD', matchC).encode('ascii', 'ignore').decode('utf-8')
        matchD = unicodedata.normalize('NFKD', matchD).encode('ascii', 'ignore').decode('utf-8')
        
        qadata = {}
        qadata['source'] = 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education'
        qadata['year'] = 2018
        qadata['type'] = 'study guide'
        qadata['qid'] = matchQN
        qadata['question'] = {"stem": matchQ.replace('\n',' ').strip(),
                              "choices": [
                                  {"label": "A", "text": matchA.replace('\n',' ').strip()},
                                  {"label": "B", "text": matchB.replace('\n',' ').strip()},
                                  {"label": "C", "text": matchC.replace('\n',' ').strip()},
                                  {"label": "D", "text": matchD.replace('\n',' ').strip()}]}
    
        
        #if matchQN in answer_dict.keys():
        qadata['answerKey'] = answer_dict.get(matchQN)
        
        statements = []
        for choice, text in zip(['A', 'B', 'C', 'D'], [matchA, matchB, matchC, matchD]):
            is_correct = (answer_dict.get(matchQN) == choice)
            statements.append({
                "label": is_correct,
                "statement": matchQ.replace('\n',' ').strip() + ' ' + text.replace('\n', ' ').strip()
            })
        explanation = explanation_dict.get(matchQN)
        if explanation is not None:
            exp = unicodedata.normalize('NFKD', explanation).encode('ascii', 'ignore').decode('utf-8')
            qadata['explanation'] = exp
        else:
            qadata['explanation'] = None
        qadata['statements'] = statements
        pmpqa.append(qadata)

pmpqa


[{'source': 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education',
  'year': 2018,
  'type': 'study guide',
  'qid': '1',
  'question': {'stem': 'You are the project manager of the GHY Project for your organization. A portion of this project includes dangerous work; although members of the project team could likely complete the work, your sponsor doesnt want to accept the risk. In this situation, which of the following may be used as a risk mitigation tool?',
   'choices': [{'label': 'A', 'text': 'A vendor proposal'},
    {'label': 'B', 'text': 'A contract'},
    {'label': 'C', 'text': 'A quotation'},
    {'label': 'D', 'text': 'Project requirements'}]},
  'answerKey': 'B',
  'explanation': 'A contract can be used as a risk mitigation tool. Procurement of risky activities is known\nas transferencethe risk does not disappear, but the responsibility for the risk is transferred to\nthe vendor. A, C, and D are incorrect. A vendor proposal, a quotation, a

In [7]:
# save the PMP-QA dataset as jsonl
with open(output_file, 'w') as fout:
    for dic in pmpqa:
        fout.write(json.dumps(dic) + '\n')

In [1]:
# import libraries
import re
import pdfplumber
import json

In [2]:
# define PDF file path
file_name = 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education (2018)'
input_file = file_name + '.pdf'
output_file = file_name + '_chapter13_statements.jsonl'

In [3]:
pages_to_extract = list(range(740, 753))

    
all_text = ""

with pdfplumber.open(input_file) as pdf:
    for page_num, page in enumerate(pdf.pages):
        if page_num + 1 in pages_to_extract:
            page_text = page.extract_text()
            if page_text:
                all_text += '\n' + page_text
pattern1 = r"CertPrs_2015/PMP® Project Management Professional Study Guide/Joseph Phillips/86198-8/Chapter \d+"
all_text=re.sub(pattern1, '', all_text)
pattern2 = r"Self Test Answers \d+"
all_text=re.sub(pattern2, '', all_text)
pattern3 = r"SELF TEST ANSWERS"
all_text=re.sub(pattern3, '', all_text)
pattern4 = r"\d+ Chapter 13 Introducing Project Stakeholder Management"
all_text=re.sub(pattern4, '', all_text)
pattern5 =r"13-ch13.indd \d+ 02/11/17 2:36 PM"
all_text=re.sub(pattern5, '', all_text)   
#print(all_text)

In [4]:
section = r"^\d+\.\s+([\S\s]*?)(?=\Z|^\d+\.\s+)"
# capture sections from question to question
sections = re.finditer(section, all_text, re.MULTILINE)

qnumber = r"(^\d+)\.\s+[\S\s]"
qnumbers = re.finditer(qnumber, all_text, re.MULTILINE)

question = r"\d+\.\s+([\S\s]*?)\nA\."
questions = re.finditer(question, all_text, re.MULTILINE)

choiceA = r"\nA\.([\S\s]*?)\nB\."
choiceAs = re.finditer(choiceA, all_text, re.MULTILINE)

choiceB = r"\nB\.([\S\s]*?)\nC\."
choiceBs = re.finditer(choiceB, all_text, re.MULTILINE)

choiceC = r"\nC\.([\S\s]*?)\nD\."
choiceCs = re.finditer(choiceC, all_text, re.MULTILINE)

choiceD = r"\nD\.([\S\s]*?)\n"
choiceDs = re.finditer(choiceD, all_text, re.MULTILINE)

answer_dict={}
    
answer_key_regex = r"☑\s*([ABCD])\.\s*(.*?)\n"

matches = re.findall(answer_key_regex, all_text, re.DOTALL)

i=1
for match in matches:
    answer = match[0]
    explanation = match[1].strip()
    answer_dict[str(i)]=answer
    i+=1
answer_dict

{'1': 'B',
 '2': 'D',
 '3': 'A',
 '4': 'B',
 '5': 'A',
 '6': 'B',
 '7': 'A',
 '8': 'B',
 '9': 'C',
 '10': 'C',
 '11': 'B',
 '12': 'D',
 '13': 'A',
 '14': 'B',
 '15': 'D',
 '16': 'D',
 '17': 'D',
 '18': 'A',
 '19': 'D',
 '20': 'C',
 '21': 'D',
 '22': 'A',
 '23': 'D',
 '24': 'A',
 '25': 'A'}

In [5]:
explanation_dict={}
explanation_regex = r"☑\s*[ABCD]\.\s*(.*?)\n(?:☐✗\n(.*?\n)|$)"

matches = re.findall(explanation_regex, all_text, re.DOTALL)
i=1
for match in matches:
    explanation_correct = match[0].strip()
    explanation_incorrect = match[1].strip() if match[1] else ""
    explanation_dict[str(i)]=f"{explanation_correct} {explanation_incorrect}"
    i+=1
explanation_dict

{'1': 'Negative stakeholders are people who do not want your project to succeed or even exist\nin the organization. A, C, and D are incorrect. A is incorrect because uninformed describes a stakeholder who',
 '2': 'Beth will not need the risk management plan as part of the stakeholder identification\nprocess. A, B, and C are incorrect. These are the correct inputs for stakeholder identification. Note',
 '3': 'There are three logical steps to stakeholder analysis: First you need to identify the project\nstakeholders. Next, you’ll prioritize the stakeholders based on their role and influence in the\nproject. Finally, you’ll anticipate stakeholder responses to issues, concerns, and requirements in\nthe project. B, C, and D are incorrect. These answers do not reflect the correct order of activities in',
 '4': 'A project that has this many stakeholders is likely to create groups of stakeholders\nto manage. For example, the stakeholders could be grouped by departments, roles in the\norganizat

In [6]:
import re
import unicodedata  # Module Unicode

matchesS = re.finditer(section, all_text, re.MULTILINE)
pmpqa = []

for sectionNum, sectionMatch in enumerate(matchesS):
    matchesQN = re.findall(qnumber, sectionMatch.group(0), re.MULTILINE)
    matchesQ = re.findall(question, sectionMatch.group(0), re.MULTILINE)
    matchesA = re.findall(choiceA, sectionMatch.group(0), re.MULTILINE)
    matchesB = re.findall(choiceB, sectionMatch.group(0), re.MULTILINE)
    matchesC = re.findall(choiceC, sectionMatch.group(0), re.MULTILINE)
    matchesD = re.findall(choiceD, sectionMatch.group(0), re.MULTILINE)
    
    for matchQN, matchQ, matchA, matchB, matchC, matchD in zip(matchesQN, matchesQ, matchesA, matchesB, matchesC, matchesD):
        # Normalize Unicode characters to their closest ASCII representation
        # This removes accents and converts special characters like '\u2018' and '\u2019' to standard ASCII equivalents.
        matchQ = unicodedata.normalize('NFKD', matchQ).encode('ascii', 'ignore').decode('utf-8')
        matchA = unicodedata.normalize('NFKD', matchA).encode('ascii', 'ignore').decode('utf-8')
        matchB = unicodedata.normalize('NFKD', matchB).encode('ascii', 'ignore').decode('utf-8')
        matchC = unicodedata.normalize('NFKD', matchC).encode('ascii', 'ignore').decode('utf-8')
        matchD = unicodedata.normalize('NFKD', matchD).encode('ascii', 'ignore').decode('utf-8')
        
        qadata = {}
        qadata['source'] = 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education'
        qadata['year'] = 2018
        qadata['type'] = 'study guide'
        qadata['qid'] = matchQN
        qadata['question'] = {"stem": matchQ.replace('\n',' ').strip(),
                              "choices": [
                                  {"label": "A", "text": matchA.replace('\n',' ').strip()},
                                  {"label": "B", "text": matchB.replace('\n',' ').strip()},
                                  {"label": "C", "text": matchC.replace('\n',' ').strip()},
                                  {"label": "D", "text": matchD.replace('\n',' ').strip()}]}
    
        
        #if matchQN in answer_dict.keys():
        qadata['answerKey'] = answer_dict.get(matchQN)
        
        statements = []
        for choice, text in zip(['A', 'B', 'C', 'D'], [matchA, matchB, matchC, matchD]):
            is_correct = (answer_dict.get(matchQN) == choice)
            statements.append({
                "label": is_correct,
                "statement": matchQ.replace('\n',' ').strip() + ' ' + text.replace('\n', ' ').strip()
            })
        explanation = explanation_dict.get(matchQN)
        if explanation is not None:
            exp = unicodedata.normalize('NFKD', explanation).encode('ascii', 'ignore').decode('utf-8')
            qadata['explanation'] = exp
        else:
            qadata['explanation'] = None
        qadata['statements'] = statements
        pmpqa.append(qadata)

pmpqa


[{'source': 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education',
  'year': 2018,
  'type': 'study guide',
  'qid': '25',
  'question': {'stem': 'You are the project manager of a new software development project in your company. Your company operates in a matrix environment and utilizes a project management office to structure projects. This current project has 78 stakeholders and will last for 18 months. Sam, one of the project stakeholders, informs you that two of his employees will be leaving the organization and will no longer be available as resources on your project. In addition to the project staffing management plan, what other document should you update?',
   'choices': [{'label': 'A', 'text': 'Stakeholder register'},
    {'label': 'B', 'text': 'Risk register'},
    {'label': 'C', 'text': 'Change log'},
    {'label': 'D', 'text': 'Project schedule'}]},
  'answerKey': 'A',
  'explanation': 'The stakeholder register is the best answer, becaus

In [7]:
# save the PMP-QA dataset as jsonl
with open(output_file, 'w') as fout:
    for dic in pmpqa:
        fout.write(json.dumps(dic) + '\n')

In [1]:
# import libraries
import re
import pdfplumber
import json

In [2]:
# define PDF file path
file_name = 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education (2018)'
input_file = file_name + '.pdf'
output_file = file_name + '_chapter14_statements.jsonl'

In [3]:
pages_to_extract = list(range(766, 777))

    
all_text = ""

with pdfplumber.open(input_file) as pdf:
    for page_num, page in enumerate(pdf.pages):
        if page_num + 1 in pages_to_extract:
            page_text = page.extract_text()
            if page_text:
                all_text += '\n' + page_text
pattern1 = r"CertPrs_2015/PMP® Project Management Professional Study Guide/Joseph Phillips/86198-8/Chapter \d+"
all_text=re.sub(pattern1, '', all_text)
pattern2 = r"Self Test Answers \d+"
all_text=re.sub(pattern2, '', all_text)
pattern3 = r"SELF TEST ANSWERS"
all_text=re.sub(pattern3, '', all_text)
pattern4 = r"\d+ Chapter 14 The PMI Code of Ethics and Professional Conduct"
all_text=re.sub(pattern4, '', all_text)
pattern5 =r"14-ch14.indd \d+ 02/11/17 2:41 PM"
all_text=re.sub(pattern5, '', all_text)   
#print(all_text)

In [4]:
section = r"^\d+\.\s+([\S\s]*?)(?=\Z|^\d+\.\s+)"
# capture sections from question to question
sections = re.finditer(section, all_text, re.MULTILINE)

qnumber = r"(^\d+)\.\s+[\S\s]"
qnumbers = re.finditer(qnumber, all_text, re.MULTILINE)

question = r"\d+\.\s+([\S\s]*?)\nA\."
questions = re.finditer(question, all_text, re.MULTILINE)

choiceA = r"\nA\.([\S\s]*?)\nB\."
choiceAs = re.finditer(choiceA, all_text, re.MULTILINE)

choiceB = r"\nB\.([\S\s]*?)\nC\."
choiceBs = re.finditer(choiceB, all_text, re.MULTILINE)

choiceC = r"\nC\.([\S\s]*?)\nD\."
choiceCs = re.finditer(choiceC, all_text, re.MULTILINE)

choiceD = r"\nD\.([\S\s]*?)\n"
choiceDs = re.finditer(choiceD, all_text, re.MULTILINE)

answer_dict={}
    
answer_key_regex = r"☑\s*([ABCD])\.\s*(.*?)\n"

matches = re.findall(answer_key_regex, all_text, re.DOTALL)

i=1
for match in matches:
    answer = match[0]
    explanation = match[1].strip()
    answer_dict[str(i)]=answer
    i+=1
answer_dict

{'1': 'D',
 '2': 'C',
 '3': 'C',
 '4': 'B',
 '5': 'B',
 '6': 'A',
 '7': 'B',
 '8': 'D',
 '9': 'A',
 '10': 'C',
 '11': 'C',
 '12': 'C',
 '13': 'D',
 '14': 'B',
 '15': 'B',
 '16': 'C',
 '17': 'A',
 '18': 'A',
 '19': 'D',
 '20': 'C',
 '21': 'A',
 '22': 'C',
 '23': 'B',
 '24': 'C',
 '25': 'D'}

In [5]:
explanation_dict={}
explanation_regex = r"☑\s*[ABCD]\.\s*(.*?)\n(?:☐✗\n(.*?\n)|$)"

matches = re.findall(explanation_regex, all_text, re.DOTALL)
i=1
for match in matches:
    explanation_correct = match[0].strip()
    explanation_incorrect = match[1].strip() if match[1] else ""
    explanation_dict[str(i)]=f"{explanation_correct} {explanation_incorrect}"
    i+=1
explanation_dict

{'1': 'It would be inappropriate to bloat the project costs by 25 percent. A risk assessment\ndescribing how the project may fail if the budget is not accurate is most appropriate. A, B, and C are incorrect. They are ethically wrong. The PMP should always provide honest',
 '2': 'You should first confirm what the local practices and customs call for in regard to hiring\nfamily members before others. A, B, and D are incorrect. A and D are incorrect because they do not consider the',
 '3': 'The company policies and procedures should guide you regarding the decisions you make\nin the foreign country. A, B, and D are incorrect. A and B are incorrect because these documents are essential but',
 '4': 'This situation calls for the project team member to be reported to his manager for\ndisciplinary action. A, C, and D are incorrect. A is inappropriate because the project manager may not have',
 '5': 'The project manager must respect the authority of the functional manager. A, C, and D are incor

In [6]:
import re
import unicodedata  # Module Unicode

matchesS = re.finditer(section, all_text, re.MULTILINE)
pmpqa = []

for sectionNum, sectionMatch in enumerate(matchesS):
    matchesQN = re.findall(qnumber, sectionMatch.group(0), re.MULTILINE)
    matchesQ = re.findall(question, sectionMatch.group(0), re.MULTILINE)
    matchesA = re.findall(choiceA, sectionMatch.group(0), re.MULTILINE)
    matchesB = re.findall(choiceB, sectionMatch.group(0), re.MULTILINE)
    matchesC = re.findall(choiceC, sectionMatch.group(0), re.MULTILINE)
    matchesD = re.findall(choiceD, sectionMatch.group(0), re.MULTILINE)
    
    for matchQN, matchQ, matchA, matchB, matchC, matchD in zip(matchesQN, matchesQ, matchesA, matchesB, matchesC, matchesD):
        # Normalize Unicode characters to their closest ASCII representation
        # This removes accents and converts special characters like '\u2018' and '\u2019' to standard ASCII equivalents.
        matchQ = unicodedata.normalize('NFKD', matchQ).encode('ascii', 'ignore').decode('utf-8')
        matchA = unicodedata.normalize('NFKD', matchA).encode('ascii', 'ignore').decode('utf-8')
        matchB = unicodedata.normalize('NFKD', matchB).encode('ascii', 'ignore').decode('utf-8')
        matchC = unicodedata.normalize('NFKD', matchC).encode('ascii', 'ignore').decode('utf-8')
        matchD = unicodedata.normalize('NFKD', matchD).encode('ascii', 'ignore').decode('utf-8')
        
        qadata = {}
        qadata['source'] = 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education'
        qadata['year'] = 2018
        qadata['type'] = 'study guide'
        qadata['qid'] = matchQN
        qadata['question'] = {"stem": matchQ.replace('\n',' ').strip(),
                              "choices": [
                                  {"label": "A", "text": matchA.replace('\n',' ').strip()},
                                  {"label": "B", "text": matchB.replace('\n',' ').strip()},
                                  {"label": "C", "text": matchC.replace('\n',' ').strip()},
                                  {"label": "D", "text": matchD.replace('\n',' ').strip()}]}
    
        
        #if matchQN in answer_dict.keys():
        qadata['answerKey'] = answer_dict.get(matchQN)
        
        statements = []
        for choice, text in zip(['A', 'B', 'C', 'D'], [matchA, matchB, matchC, matchD]):
            is_correct = (answer_dict.get(matchQN) == choice)
            statements.append({
                "label": is_correct,
                "statement": matchQ.replace('\n',' ').strip() + ' ' + text.replace('\n', ' ').strip()
            })
        explanation = explanation_dict.get(matchQN)
        if explanation is not None:
            exp = unicodedata.normalize('NFKD', explanation).encode('ascii', 'ignore').decode('utf-8')
            qadata['explanation'] = exp
        else:
            qadata['explanation'] = None
        qadata['statements'] = statements
        pmpqa.append(qadata)

pmpqa


[{'source': 'Joseph Phillips - PMP Project Management Professional Study Guide-McGraw-Hill Education',
  'year': 2018,
  'type': 'study guide',
  'qid': '1',
  'question': {'stem': 'You are the project manager of the JKN Project. The project customer has requested that you inflate your cost estimates by 25 percent. He reports that his management always reduces the cost of the estimates, so this is the only way to get the monies needed to complete the project. Which of the following is the best response to this situation?',
   'choices': [{'label': 'A',
     'text': 'Do as the customer asked to ensure the project requirements can be met by adding the increase as a contingency reserve.'},
    {'label': 'B',
     'text': 'Do as the customer asked to ensure the project requirements can be met by adding the increase across each task.'},
    {'label': 'C',
     'text': 'Do as the customer asked by creating an estimate for the customers management and another for the actual project implementa

In [7]:
# save the PMP-QA dataset as jsonl
with open(output_file, 'w') as fout:
    for dic in pmpqa:
        fout.write(json.dumps(dic) + '\n')